# Imports/Pick Analyses to run

In [ ]:
import warnings

import mne
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import json
import numpy as np
import itertools
import pandas as pd
from collections import defaultdict
from collections import Counter
import re
import seaborn as sns
from statsmodels.formula.api import mixedlm, ols
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import statsmodels.api as sm
from nilearn.glm.first_level import make_first_level_design_matrix

from preprocess_nirs import *
from nirs_functions import *

from mne_nirs.channels import picks_pair_to_idx, get_long_channels
from mne_nirs.preprocessing import peak_power, scalp_coupling_index_windowed
from mne.preprocessing.nirs import source_detector_distances, scalp_coupling_index
from mne_connectivity import spectral_connectivity_time
from mne_connectivity.viz import plot_connectivity_circle
from mne.viz import circular_layout
from mne_nirs.statistics import run_glm, statsmodels_to_results

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut, cross_validate
from sklearn.pipeline import make_pipeline

import tensorflow as tf

# Set the image dpi
dpi = 600

# Set the number of cores to use
n_jobs = 1
if os.cpu_count() is not None:
    n_jobs = int(np.ceil(os.cpu_count() * 0.75))

seed_value = 42
# Set the random seed
os.environ['PYTHONHASHSEED'] = str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)
tf.config.experimental.enable_op_determinism()

# Get the mappings of the brain regions
mappings = json.load(open("processed_data/mappings/mappings.json"))

# Set thresholds for the data
sci_threshold = 0.5
cv_threshold = 15
good_threshold = 0.7
peak_power_threshold = 0.1
behavioural_accuracy_threshold = 0.6
max_times = 104

# Preprocess the data
process_data = False

# Get the participant info plots
get_participant_age_plot = False
get_meas_dates_plot = False
get_short_chns_plot = False
get_behavioural_responses = False
no_response_counts = False
run_behavioural_responses_stats = False
get_behavioural_responses_plots = False

# Get the mappings
get_mappings = False

# Get the non windowed SCI and CV measure
get_full_sci = False
get_full_cv = False

# Get the peak power/scalp coupling index dataframes
get_peak_power_sci_df = False
get_good_windows_plot = False
get_slope_pp_sci_plot = False
get_head_size_vs_sci_plot = False
get_across_participant_sci_plots = False
get_participant_sci_plots = False

# Get the epochs
get_epoch_data = False

# Get the GLM data/plots
get_glm_design_matrix_plot = False
get_glm_analysis = False
get_ind_glm_plots = False
get_group_glm_plots = False
get_group_contrast_plots = False
get_sig_group_contrast_table = False

# Get the average timeseries activity
get_roi_timeseries_activity = False

# Get the ERP plots
get_erp_plots = False

# Get the connectivity data
run_ind_connectivity = False
get_condition_con_plots = False
get_variance_con_plots = False
run_group_level_t_tests = False
get_group_level_t_tests_chord_plots = False
get_group_level_t_tests_roi_chord_plots = False
get_emotion_analysis_plots = False

# Run decoding analysis
run_traditional_raw_across_decoding = False
run_dl_raw_across_decoding = True
run_traditional_con_across_decoding = False

# Get the decoding plots
get_decoding_table_scores_plots = True
get_decoding_individual_scores_plots = True

raws = []
raw_ods = []
raw_haemos = []
raw_haemo_good_recordings = []

C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nilearn\glm\__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


# Get Participants

In [6]:
# Define the folder path
data_path = os.path.join(os.getcwd(), 'data')

# Get a list of paths of all the subfolders of the folders labeled 'P_1', 'P_2', etc.
participants = [os.path.join(data_path, f) for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

participants_with_same_order = []

# remove participants P_1 to P_11 but keep P_10, and P_12 onwards
# P_1 to P_11 have the same order of faces, and P_86 and P_87 have the same order of faces
for i in range(1, 12):
    if i != 10:
        participants_with_same_order.append(os.path.join(data_path, f'P_{i}'))
        #participants.remove(os.path.join(data_path, f'P_{i}'))

participants_with_same_order.append(os.path.join(data_path, f'P_87'))
#participants.remove(os.path.join(data_path, f'P_87'))

# remove participants P_13 due to not recording
participants.remove(os.path.join(data_path, f'P_13'))

# remove participants P_50 due to ending early
participants.remove(os.path.join(data_path, f'P_50'))

# participant P_54 used their phone
participants.remove(os.path.join(data_path, f'P_54'))

# participant P_75 did not have Real_Fear
participants.remove(os.path.join(data_path, f'P_75'))

# Search recursively for the folder with the .snirf extension
fnirs_folders = []
for participant in participants:
    for root, dirs, files in os.walk(participant):
        for file in files:
            if file.endswith('.snirf'):
                fnirs_folders.append(root)
                break

# Preprocessing

In [7]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    if process_data:
        raws = []
        raw_ods = []
        raw_haemos = []

        # Load the snirf files
        for folder in fnirs_folders:
            # find all the .snirf files in the folder but get the full path
            snirf_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.snirf')]
            if len(snirf_files) == 0:
                print(f"No .snirf files found in {folder}")
                continue
            elif len(snirf_files) > 1:
                raise Exception(f"Multiple .snirf files found in {folder}")
            else:
                raw = mne.io.read_raw_snirf(snirf_files[0], optode_frame='mri', preload=True, verbose=False)
            
            # find all the 'description.json' files in the folder but get the full path
            description_files = [f for f in os.listdir(folder) if 'description.json' in f]
            if len(description_files) == 0:
                print(f"No description.json files found in {folder}")
                continue
            elif len(description_files) > 1:
                raise Exception(f"Multiple description.json files found in {folder}")
            else:
                description = json.load(open(os.path.join(folder, description_files[0])))
                # get the parent's parent folder name
                parent_folder = os.path.basename(os.path.dirname(folder))
                # find the .csv file in the parent folder
                csv_files = [f for f in os.listdir(os.path.join(data_path, parent_folder)) if f.endswith('.csv')]
                if len(csv_files) == 0:
                    print(f"No .csv files found in {os.path.join(data_path, parent_folder)}")
                elif len(csv_files) > 1:
                    raise Exception(f"Multiple .csv files found in {os.path.join(data_path, parent_folder)}")
                else:
                    # add the path to the .csv file to the description
                    description['behavioural_data'] = os.path.join(data_path, parent_folder, csv_files[0])

            distances = np.array(source_detector_distances(raw.info))
            distance_counts = {
                "short": int(sum(distances < 0.01)),
                "long": int(sum(distances >= 0.01))
            }

            description['distance_counts'] = distance_counts

            # add the description to the raw object
            raw.info['description'] = str(description)
            raws.append(raw)

        # sort the raws by the measurement date
        raws = sorted(raws, key=lambda x: x.info['meas_date'])

        i = 1
        for raw in raws:
            raw_od, raw_haemo = preprocess_data(raw)
            raw_ods.append(raw_od)
            raw_haemos.append(raw_haemo)
            i += 1

        # clear any files in each folder
        for folder in ['processed_data/raws', 'processed_data/raw_ods', 'processed_data/raw_haemos']:
            for f in os.listdir(folder):
                os.remove(os.path.join(folder, f))

        for i, (raw, raw_od, raw_haemo) in enumerate(zip(raws, raw_ods, raw_haemos), 1):
            # save raw as a fif file
            raw.save(f'processed_data/raws/raw{i}.fif', overwrite=True, verbose=False)

            # save raw_od as a fif file
            raw_od.save(f'processed_data/raw_ods/raw_od{i}.fif', overwrite=True, verbose=False)

            # save raw_haemo as a fif file
            raw_haemo.save(f'processed_data/raw_haemos/raw_haemo{i}.fif', overwrite=True, verbose=False)
        
        raws = []
        raw_ods = []
        raw_haemos = []

# Participant Information

### Participant Age

In [ ]:
if get_participant_age_plot:
    good_recordings = True
    if good_recordings:
        # Load the good recordings
        raw_haemos = load_data('raw_haemos', 'good') if len(raw_haemos) == 0 else raw_haemos
        title = "age_distribution_good"
    else:
        # Load all recordings
        raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos
        title = "age_distribution_all"

    ages = []
    for raw_haemo in raw_haemos:
        description = get_info(raw_haemo)
        age = description['age']
        ages.append(age)

    # Convert ages to numeric type
    ages = np.array(ages, dtype=float)

    # Sort the ages
    ages = sorted(ages)

    # Get the number of participants
    n_participants = len(ages)

    # Get the mean and standard deviation of the ages
    mean_age = np.mean(ages)
    std_age = np.std(ages)

    # Get the minimum and maximum ages
    min_age = min(ages)
    max_age = max(ages)
    
    # Plot the ages in a histogram
    plt.figure(figsize=(8, 6))
    plt.hist(ages, bins=30, color='blue', alpha=0.7)
    plt.xlabel('Age (years)', fontsize=14)
    plt.ylabel('Count', fontsize=14)
    plt.title(f'Age Distribution of Participants\nN ={n_participants}, Mean={mean_age:.2f}, SD={std_age:.2f}, Min={min_age}, Max={max_age}', fontsize=16)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()
    plt.savefig('plots/participants/' + title + '.png', dpi=dpi / 2)
    plt.close()

### Measurement Dates

In [3]:
if get_meas_dates_plot:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    # Extract the measurement dates
    measurement_dates = [raw_haemo.info['meas_date'] for raw_haemo in raw_haemos]

    # Convert to pandas datetime
    measurement_dates = pd.to_datetime(measurement_dates)

    # Create a plot of the measurement dates
    plt.figure(figsize=(15, 10))
    plt.plot(measurement_dates, range(1, len(measurement_dates) + 1), 'o-')
    plt.xlabel('Measurement date')
    plt.ylabel('Participant number')
    plt.title('Measurement dates of participants, N = ' + str(len(measurement_dates)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/participants/measurement_dates.png', dpi=dpi / 4)
    plt.close()

### Short Distance Channels Distribution Plot

In [4]:
if get_short_chns_plot:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    
    total_short_chns = []
    for raw_haemo in raw_haemo_good_recordings:
        num_short_chns = get_info(raw_haemo)['distance_counts']['short']
        total_short_chns.append(num_short_chns)
    total_short_chns = np.array(total_short_chns)

    # plot a pie chart of the number of short channels
    plt.figure(figsize=(7, 6))
    plt.pie(Counter(total_short_chns).values(), labels=Counter(total_short_chns).keys(), autopct='%1.1f%%')
    plt.title('Distribution of number of short channels across good recordings, N = ' + str(len(total_short_chns)))
    plt.tight_layout()
    plt.savefig('plots/participants/short_channels.png', dpi=dpi / 4)
    plt.close()

### Generate Behavioural Responses Dataframe

In [6]:
if get_behavioural_responses:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    participant_scores = pd.DataFrame()
    for i, raw_haemo in enumerate(raw_haemos, 1):
        path = get_info(raw_haemo)['behavioural_data']

        # folder_name = (P_(\d+))
        folder_name = re.search(r'P_(\d+)', path).group(0)

        # read the csv file
        behavioural_data = pd.read_csv(path)

        # only keep the first 6 columns and the column called "key_resp.keys"
        behavioural_data = behavioural_data.iloc[:, :6].join(behavioural_data[['key_resp.keys', 'key_resp.rt']])

        # remove the "blocknumber" column
        behavioural_data = behavioural_data.drop(columns='blocknumber')

        # remove the first row and reset the index
        behavioural_data = behavioural_data.iloc[1:].reset_index(drop=True)

        # divide the dataframe into rows of 9
        behavioural_data = [behavioural_data.iloc[i:i + 9] for i in range(0, len(behavioural_data), 9)]

        scores = []
        for j, data in enumerate(behavioural_data, 1):
            # in the last row, save the "key_resp.keys" column
            response = data['key_resp.keys'].iloc[-1]

            # convert the response to a boolean
            response = {'n': False, 'y': True}.get(response, np.nan)
            
            # in the last row, copy the "tasknames" and "taskmarkers" value to "filepaths" and "marker" respectively
            data.loc[data.index[-1], ["filepaths", "marker"]] = [data["tasknames"].iloc[-1], data["taskmarker"].iloc[-1]]

            # remove the "tasknames" and "taskmarker" columns
            data = data.drop(columns=["tasknames", "taskmarker"])

            # check if the (marker - 2000) in the last row was in any of the other rows
            correct_response = (data['marker'].iloc[-1] - 2000) in data['marker'].values

            # append the data to the scores list
            scores.append({
                'Participant': i,
                'Trial': j,
                'Block Name': data['blocknames'].iloc[-1],
                'File Path': data['filepaths'].iloc[-1],
                'Marker': data['marker'].iloc[-1],
                'Response Time': data['key_resp.rt'].iloc[-1],
                'Response': response,
                'Correct Response': correct_response
            })
        
        # append the scores to the participant_scores dataframe
        participant_scores = pd.concat([participant_scores, pd.DataFrame(scores)])

    # for each row, if "Response" == "Correct Response", create a new column called "Correct" and set it to True, else False, if the "Response" is NaN, set it to NaN, don't get rid of the "Correct Response" column
    participant_scores['Correct'] = participant_scores.apply(
        lambda x: np.nan if pd.isna(x['Response']) else bool(x['Response']) == bool(x['Correct Response']), axis=1
    )

    # subtract 2000 from the "Marker" column
    participant_scores['Marker'] = participant_scores['Marker'] - 2000

    # for each row in 'Marker' replace it with two columns 'Face Type' and 'Emotion' from the trigger_decoder function, which returns a tuple
    participant_scores[['Face Type', 'Emotion']] = participant_scores['Marker'].apply(trigger_decoder).apply(pd.Series)

    # remove the "Block Name", "File Path", "Marker", columns
    participant_scores = participant_scores.drop(columns=["Block Name", "File Path", "Marker"])

    # move "Face Type" and "Emotion" to after Participant
    participant_scores = participant_scores[['Participant', 'Trial', 'Face Type', 'Emotion', 'Response Time', 'Response', 'Correct Response', 'Correct']]

    # save the participant_scores dataframe
    participant_scores.to_csv('processed_data/behavioural_responses/participant_scores.csv', index=False)

# Load the participant scores
participant_scores = pd.read_csv('processed_data/behavioural_responses/participant_scores.csv')

# get and save the original participant scores before dropping any participants
participant_scores_original = participant_scores.copy()

# get the number of Correct responses per participant
correct_responses = participant_scores.groupby('Participant')['Correct'].sum()

# get the indices of the participants with > 60% correct responses
good_recordings = correct_responses[correct_responses > 56 * behavioural_accuracy_threshold].index

# only keep the participants with > accuracy_threshold correct responses
participant_scores = participant_scores[participant_scores['Participant'].isin(good_recordings)]

In [ ]:
if no_response_counts:
    # filter participant_scores to only include the trials with a nan value in the 'Response' column
    no_response = participant_scores[participant_scores['Response'].isna()]

    # count the number of rows per participant
    no_response_counts = no_response['Participant'].value_counts()

    # make a histogram of the number of trials with no response per participant
    plt.figure(figsize=(10, 6))
    bins = range(1, no_response_counts.values.max() + 2)
    n, bins, patches = plt.hist(no_response_counts.values, bins=bins, color='gold', alpha=0.7, edgecolor='black')

    # Apply a gradient color to each bar using a colormap
    cmap = plt.cm.Wistia
    for i, patch in enumerate(patches):
        color = cmap(i / len(patches))
        patch.set_facecolor(color)

    plt.xlabel('Number of Blocks with No Response', fontsize=14)
    plt.ylabel('Number of Participants', fontsize=14)
    plt.title('Distribution of Blocks with No Response per Participant', fontsize=16)
    plt.xticks(
        ticks=[b + 0.5 for b in bins[:-1]],
        labels=[str(b) for b in bins[:-1]]
    )
    plt.tight_layout(pad=0)
    plt.savefig('plots/behavioural_responses/no_response_counts.png', dpi=dpi / 2)
    plt.close()

#### Behavioural Responses Statistics

In [ ]:
if run_behavioural_responses_stats:
    # Create a copy of the 'Correct' column and convert it to 1 if True, 0 if False
    participant_scores['Correct Numeric'] = np.where(participant_scores['Correct'] == True, 1, 0)
    participant_scores_original['Correct Numeric'] = np.where(participant_scores_original['Correct'] == True, 1, 0)

    # Create a copy of the relevant columns
    data_copy = participant_scores[['Face Type', 'Emotion', 'Correct Numeric']].copy()

    # Rename the column for consistency
    data_copy = data_copy.rename(columns={'Face Type': 'Face_Type', 'Correct Numeric': 'Correct_Numeric'})

    # Build an OLS model including Face Type, Emotion, and their interaction
    model = ols("Correct_Numeric ~ C(Face_Type) * C(Emotion)", data=data_copy).fit()

    # Perform two-way ANOVA
    anova_table = sm.stats.anova_lm(model, typ=3)  # Type 3 ANOVA

    # Print the ANOVA table with p-values not in scientific notation
    anova_table['PR(>F)'] = anova_table['PR(>F)'].apply(lambda x: f"{x:.6f}" if not pd.isnull(x) else x)

    # save the anova table
    anova_table.to_csv('processed_data/behavioural_responses/anova_table.csv')

    p_value_face_type = anova_table.loc['C(Face_Type)', 'PR(>F)']
    p_value_emotion = anova_table.loc['C(Emotion)', 'PR(>F)']
    p_value_interaction = anova_table.loc['C(Face_Type):C(Emotion)', 'PR(>F)']

    # Check if there is a significant correlation between Response Time and Correct
    # Ensure both columns have the same length by dropping NaN values in both simultaneously
    valid_data = participant_scores[['Response Time', 'Correct Numeric']].dropna()
    correlation, p_value = stats.pearsonr(valid_data['Response Time'], valid_data['Correct Numeric'])

#### Behavioural Responses Plots

In [23]:
if get_behavioural_responses_plots:
    # get the sum of correct responses for each participant
    participant_correct_responses = participant_scores.groupby('Participant')['Correct'].sum()
    participant_correct_responses_original = participant_scores_original.groupby('Participant')['Correct'].sum()

    n_blocks = len(participant_scores) / len(participant_scores['Participant'].unique())
    n_blocks_original = len(participant_scores_original) / len(participant_scores_original['Participant'].unique())

    # plot this data
    plt.figure(figsize=(12, 6))
    plt.bar(participant_correct_responses_original.index, participant_correct_responses_original.values, width=0.9)
    plt.axhline(y=n_blocks_original * behavioural_accuracy_threshold, color='r', linestyle='--', label='Accuracy Threshold')
    plt.xlabel('Participant', fontsize=16)
    plt.ylabel('Number of Correct Responses', fontsize=16)
    plt.title('Number of Correct Responses per Participant\nMean across all ' + str(len(participant_correct_responses_original)) + ' participants: ' + str(round(participant_correct_responses_original.mean() / n_blocks_original * 100, 2)) + '%', fontsize=24)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_participant.png', dpi=dpi // 2)
    plt.close()

    # get the percentage of correct responses for each type of face and emotion and the combination of both
    face_type_correct_responses = participant_scores.groupby('Face Type')['Correct'].mean() * 100
    emotion_correct_responses = participant_scores.groupby('Emotion')['Correct'].mean() * 100
    face_type_emotion_correct_responses = participant_scores.groupby(['Face Type', 'Emotion'])['Correct'].mean() * 100

    # Create a figure and specify a 2x2 GridSpec
    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(2, 2)  # 2 rows, 2 columns

    plt.suptitle('Correct Memory Task Responses by Face Type and Emotion', fontsize=16)

    # Top-left: Face Type pie chart
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.pie(face_type_correct_responses,
            labels=face_type_correct_responses.index,
            autopct='%1.1f%%', startangle=90, radius=1)
    ax1.set_title('Face Type\np = ' + str(p_value_face_type))

    # Top-right: Emotion pie chart
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.pie(emotion_correct_responses,
            labels=emotion_correct_responses.index,
            autopct='%1.1f%%', startangle=90, radius=1)
    ax2.set_title('Emotion\np = ' + str(p_value_emotion))

    # Bottom row spanning both columns: Face Type & Emotion pie chart
    ax3 = fig.add_subplot(gs[1, :])  # span columns 0 and 1
    ax3.pie(face_type_emotion_correct_responses,
            labels=[f"{ft}-{em}" for ft, em in face_type_emotion_correct_responses.index],
            autopct='%1.1f%%', startangle=90, radius=1)
    ax3.set_title('Face Type & Emotion\np = ' + str(p_value_interaction))

    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_face_type_emotion.png', dpi=dpi / 4)
    plt.close()

    # get the average number of correct responses for each trial
    trial_correct_responses = participant_scores.groupby('Trial')['Correct'].mean().to_list()

    # calculate the regression line using stats.linregress
    x = range(1, len(trial_correct_responses) + 1)
    y = trial_correct_responses
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # plot this data
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(trial_correct_responses) + 1), trial_correct_responses, 'o-')
    plt.plot(x, [slope * i + intercept for i in x], 'r--')
    plt.xlabel('Trial')
    plt.ylabel('Percentage of Correct Responses')
    plt.title('Percentage of Correct Responses per Trial, R^2 = ' + str(round(r_value ** 2, 2)) + ', p = ' + str(round(p_value, 4)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_trial.png', dpi=dpi / 4)
    plt.close()

    # get the average response time for each trial
    trial_response_times = participant_scores.groupby('Trial')['Response Time'].mean().to_list()
    
    # calculate the regression line using stats.linregress
    x = range(1, len(trial_response_times) + 1)
    y = trial_response_times
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    # plot this data
    plt.figure(figsize=(10, 10))
    plt.plot(range(1, len(trial_response_times) + 1), trial_response_times, 'o-')
    plt.plot(x, [slope * i + intercept for i in x], 'r--')
    plt.xlabel('Trial')
    plt.ylabel('Response Time (s)')
    plt.title('Response Time per Trial, R^2 = ' + str(round(r_value ** 2, 2)) + ', p = ' + str(p_value) + '\nCorrelation with Correct answer: ' + str(round(correlation, 4)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/response_times_by_trial.png', dpi=dpi)
    plt.close()

# Mapping brain regions

In [2]:
if get_mappings:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    # get the channel names for hbo
    ch_names_hbo = [ch_name for ch_name in raw_haemos[0].ch_names if 'hbo' in ch_name]

    ch_mapping_hbo = {
        "Left Frontal": [],
        "Right Frontal": [],
        "Left Prefrontal": [],
        "Right Prefrontal": [],
        "Left Parietal": [],
        "Right Parietal": [],
        "Left Occipital": [],
        "Right Occipital": []
    }

    group_boundaries = [0]

    ch_mapping_hbo["Left Frontal"].append('S1_D1 hbo')

    ch_mapping_hbo["Left Frontal"].append('S1_D2 hbo')

    ch_mapping_hbo["Left Frontal"].append('S6_D3 hbo')

    ch_mapping_hbo["Left Frontal"].append('S6_D31 hbo')

    # find the channels that have 'S2_', 'S3_', 'S4_', 'S5_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S2_' in ch_name or 'S3_' in ch_name or 'S4_' in ch_name or 'S5_' in ch_name]:
        ch_mapping_hbo["Left Frontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Frontal"]))

    ch_mapping_hbo["Right Frontal"].append('S1_D17 hbo')

    ch_mapping_hbo["Right Frontal"].append('S6_D2 hbo')

    ch_mapping_hbo["Right Frontal"].append('S6_D18 hbo')

    # find the channels that have 'S9_', 'S10_', 'S11_', 'S12_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S9_' in ch_name or 'S10_' in ch_name or 'S11_' in ch_name or 'S12_' in ch_name]:
        ch_mapping_hbo["Right Frontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Frontal"]) + group_boundaries[-1])

    # find the channels that have 'S7_', 'S8_', 'S25_', 'S26_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S7_' in ch_name or 'S8_' in ch_name or 'S25_' in ch_name or 'S26_' in ch_name]:
        ch_mapping_hbo["Left Prefrontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Prefrontal"]) + group_boundaries[-1])

    # find the channels that have 'S13_', 'S14_', 'S15_', 'S16_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S13_' in ch_name or 'S14_' in ch_name or 'S15_' in ch_name or 'S16_' in ch_name]:
        ch_mapping_hbo["Right Prefrontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Prefrontal"]) + group_boundaries[-1])

    # find the channels that have 'S27_', 'S28_', 'S29_', 'S30_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S27_' in ch_name or 'S28_' in ch_name or 'S29_' in ch_name or 'S30_' in ch_name]:
        ch_mapping_hbo["Left Parietal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Parietal"]) + group_boundaries[-1])

    # find the channels that have 'S17_', 'S18_', 'S19_', 'S20_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S17_' in ch_name or 'S18_' in ch_name or 'S19_' in ch_name or 'S20_' in ch_name]:
        ch_mapping_hbo["Right Parietal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Parietal"]) + group_boundaries[-1])

    ch_mapping_hbo["Left Occipital"].append('S21_D13 hbo')

    ch_mapping_hbo["Left Occipital"].append('S21_D16 hbo')

    ch_mapping_hbo["Left Occipital"].append('S23_D15 hbo')

    # find the channels that have 'S32_', 'S31_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S32_' in ch_name or 'S31_' in ch_name]:
        ch_mapping_hbo["Left Occipital"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Occipital"]) + group_boundaries[-1])

    ch_mapping_hbo["Right Occipital"].append('S23_D16 hbo')

    ch_mapping_hbo["Right Occipital"].append('S21_D28 hbo')

    ch_mapping_hbo["Right Occipital"].append('S23_D30 hbo')

    # find the channels that have 'S22_', 'S24_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S22_' in ch_name or 'S24_' in ch_name]:
        ch_mapping_hbo["Right Occipital"].append(ch_name)

    ch_mapping_hbr = {region: [channel.replace('hbo', 'hbr') for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    ch_mapping_hbt = {region: [channel.replace('hbo', 'hbt') for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    ch_mapping_all = {region: ch_mapping_hbo[region] + ch_mapping_hbr[region] + ch_mapping_hbt[region] for region in ch_mapping_hbo}

    # concatenate the values of the dictionary into a list
    all_channels_hbo = [channel for region in ch_mapping_hbo.values() for channel in region]

    # duplicate all_channels but replace 'hbo' with 'hbr'
    all_channels_hbr = [channel.replace('hbo', 'hbr') for channel in all_channels_hbo]

    all_channels_hbt = [channel.replace('hbo', 'hbt') for channel in all_channels_hbo]

    # concatenate all_channels_hbo and all_channels_hbr
    all_channels = all_channels_hbo + all_channels_hbr + all_channels_hbt

    # make a dictionary called ch_mapping_names that has the channel names without the 'hbo' or 'hbr' at the end
    ch_mapping_names = {region: [channel[:-4] for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    # make a list of all the channel names without the 'hbo' or 'hbr' at the end
    all_channels_names = [channel[:-4] for channel in all_channels_hbo]

    ch_names_original = [ch_name[:-4] for ch_name in ch_names_hbo]

    # Collect all new variables into a dictionary
    mappings = {
        "ch_names_hbo": ch_names_hbo,
        "ch_mapping_hbo": ch_mapping_hbo,
        "ch_mapping_hbr": ch_mapping_hbr,
        "ch_mapping_hbt": ch_mapping_hbt,
        "ch_mapping_all": ch_mapping_all,
        "all_channels_hbo": all_channels_hbo,
        "all_channels_hbr": all_channels_hbr,
        "all_channels_hbt": all_channels_hbt,
        "all_channels": all_channels,
        "ch_mapping_names": ch_mapping_names,
        "all_channels_names": all_channels_names,
        "ch_names_original": ch_names_original,
        "group_boundaries": group_boundaries
    }

    # Save the dictionary to a JSON file
    with open("processed_data//mappings/mappings.json", "w") as json_file:
        json.dump(mappings, json_file, indent=4)

# Signal Quality

### Scalp Coupling Index (SCI)

In [9]:
if get_full_sci:
    raw_ods = load_data('raw_ods') if len(raw_ods) == 0 else raw_ods

    # for each recording, count the number of channels with a sci greater than good_threshold
    good_channels = [sum(scalp_coupling_index(raw_od, verbose=False) >= sci_threshold) for raw_od in raw_ods]
    bad_channels = [sum(scalp_coupling_index(raw_od, verbose=False) < sci_threshold) for raw_od in raw_ods]
    good_recordings = sum([good_channel >= good_threshold * (good_channel + bad_channel) for good_channel, bad_channel in zip(good_channels, bad_channels)])

    # Plot the good vs bad channels for each recording in a dual bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(range(len(good_channels)), good_channels, label='Good Channels')
    ax.bar(range(len(bad_channels)), bad_channels, bottom=good_channels, label='Bad Channels')
    ax.set_xlabel('Recording')
    ax.set_ylabel('Number of Channels')
    ax.axhline(raw_ods[0].info['nchan'] * good_threshold, color='green', linestyle='--')
    title = f'Good vs Bad Channels (T = {sci_threshold})\nGood Recordings: {good_recordings}, N = {len(raw_ods)}, Retention Rate: {good_recordings / len(raw_ods) * 100:.2f}%'
    ax.set_title(title)
    ax.legend()
    plt.savefig(f'plots/signal quality/Signal Quality (SCI).png', dpi=dpi)
    plt.close()

### Coefficient of Variance (CV)

In [11]:
if get_full_cv:
    raws = load_data('raws') if len(raws) == 0 else raws

    # for each recording, count the number of channels with a cv less than cov_threshold
    good_channels = [sum(100 * np.std(ch) / np.mean(ch) < cv_threshold for ch in get_long_channels(raw).get_data()) for raw in raws]
    bad_channels = [sum(100 * np.std(ch) / np.mean(ch) >= cv_threshold for ch in get_long_channels(raw).get_data()) for raw in raws]
    good_recordings = sum([good_channel >= good_threshold * (good_channel + bad_channel) for good_channel, bad_channel in zip(good_channels, bad_channels)])

    # Plot the good vs bad channels for each recording in a dual bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(range(len(good_channels)), good_channels, label='Good Channels')
    ax.bar(range(len(bad_channels)), bad_channels, bottom=good_channels, label='Bad Channels')
    ax.set_xlabel('Recording')
    ax.set_ylabel('Number of Channels')
    ax.axhline(raws[0].info['nchan'] * good_threshold, color='green', linestyle='--')
    title = f'Good vs Bad Channels (T = {cv_threshold})\nGood Recordings: {good_recordings}, N = {len(raws)}, Retention Rate: {good_recordings / len(raws) * 100:.2f}%'
    ax.set_title(title)
    ax.legend()
    plt.savefig(f'plots/signal quality/Signal Quality (CV).png', dpi=dpi)
    plt.close()

### Get Peak Power/SCI Sliding Window CSV

In [12]:
if get_peak_power_sci_df:
    raw_ods = load_data('raw_ods') if len(raw_ods) == 0 else raw_ods

    peak_power_df = pd.DataFrame()
    sci_df = pd.DataFrame()
    for i, raw_od in enumerate(raw_ods, 1):
        raw_od_annotated_pp, scores_pp, times_pp = peak_power(raw_od, time_window=5, verbose=False)
        raw_od_annotated_sci, scores_sci, times_sci = scalp_coupling_index_windowed(raw_od, time_window=5, verbose=False)

        # Convert scores and times to a DataFrame
        df_pp = pd.DataFrame(scores_pp.T, columns=[ch_name for ch_name in raw_od.ch_names])
        df_sci = pd.DataFrame(scores_sci.T, columns=[ch_name for ch_name in raw_od.ch_names])

        # Add time window information
        df_pp["Start_Time"] = [t[0] for t in times_pp]
        df_pp["End_Time"] = [t[1] for t in times_pp]
        df_sci["Start_Time"] = [t[0] for t in times_sci]
        df_sci["End_Time"] = [t[1] for t in times_sci]

        # Add an index column for window number
        df_pp.insert(0, 'Window', range(1, len(df_pp) + 1))
        df_sci.insert(0, 'Window', range(1, len(df_sci) + 1))

        # Reorder columns so time comes first
        df_pp = df_pp[["Start_Time", "End_Time"] + list(df_pp.columns[:-2])]
        df_sci = df_sci[["Start_Time", "End_Time"] + list(df_sci.columns[:-2])]

        # remove the columns with '850' in the name
        df_pp = df_pp.loc[:, ~df_pp.columns.str.contains('850')]
        df_sci = df_sci.loc[:, ~df_sci.columns.str.contains('850')]

        # rename the columns to remove the ' 760' at the end if it exists
        df_pp.columns = [col[:-4] if col.endswith(' 760') else col for col in df_pp.columns]
        df_sci.columns = [col[:-4] if col.endswith(' 760') else col for col in df_sci.columns]

        # Add a column for participant number, make it the first column
        df_pp.insert(0, 'Participant', i)
        df_sci.insert(0, 'Participant', i)

        # Append the DataFrame to the list
        peak_power_df = pd.concat([peak_power_df, df_pp])
        sci_df = pd.concat([sci_df, df_sci])

    # reset the index
    peak_power_df.reset_index(drop=True, inplace=True)
    sci_df.reset_index(drop=True, inplace=True)

    # Save the DataFrame to a CSV file
    peak_power_df.to_csv('processed_data/windows/peak_power.csv', index=False)
    sci_df.to_csv('processed_data/windows/sci.csv', index=False)

# Load the DataFrame
peak_power_df = pd.read_csv('processed_data/windows/peak_power.csv')
sci_df = pd.read_csv('processed_data/windows/sci.csv')

### Peak Power/SCI Sliding Window

In [13]:
if get_good_windows_plot:
    # Compute the proportion of windows with peak power > peak_power_threshold for each channel
    percentage_good_windows_peak_power_df = (
        peak_power_df.groupby("Participant")[peak_power_df.columns[4:]]
        .apply(lambda df: (df > peak_power_threshold).sum() / len(df))
    )

    # add a good recordings column
    good_recordings = (percentage_good_windows_peak_power_df > good_threshold).sum(axis=1) / len(percentage_good_windows_peak_power_df.columns)
    percentage_good_windows_peak_power_df.insert(0, f'Good Recordings (peak_power > {peak_power_threshold} for > {good_threshold * 100}% of channels)', good_recordings)

    # Compute the proportion of windows with SCI > good_threshold for each channel
    percentage_good_windows_sci_df = (
        sci_df.groupby("Participant")[sci_df.columns[4:]]
        .apply(lambda df: (df > sci_threshold).sum() / len(df))
    )

    # add a good recordings column
    good_recordings = (percentage_good_windows_sci_df > good_threshold).sum(axis=1) / len(percentage_good_windows_sci_df.columns)
    percentage_good_windows_sci_df.insert(0, f'Good Recordings (SCI > {sci_threshold} for > {good_threshold * 100}% of channels)', good_recordings)

    # merge the two dataframes on the first 2 columns
    percentage_good_windows_df = pd.merge(percentage_good_windows_peak_power_df[percentage_good_windows_peak_power_df.columns[:1]], percentage_good_windows_sci_df[percentage_good_windows_sci_df.columns[:1]], on='Participant')

    # create a new column that is true if both columns are greater than good_threshold
    percentage_good_windows_df['Good Recording'] = (percentage_good_windows_df.iloc[:, 0] > good_threshold) & (percentage_good_windows_df.iloc[:, 1] > good_threshold)

    # save the dataframe to a csv file
    percentage_good_windows_df.to_csv('processed_data/windows/percentage_good_windows.csv', index=False)

    # Plot a bar chart where only the SCI windows are shown for each participant
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(percentage_good_windows_df))
    ax.bar(x, percentage_good_windows_df.iloc[:, 1], label='SCI', color='#FF8C00')  # Darker orange
    ax.axhline(good_threshold, color='green', linestyle='--')
    ax.set_xlabel('Participant')
    ax.set_ylabel('Percentage of Channels (%)')
    ax.set_title(f'Percentage of Channels where SCI > ' + str(sci_threshold) + ' for > ' + str(good_threshold * 100) + f'% of windows\nGood Recordings: {percentage_good_windows_df["Good Recording"].sum()}/{len(percentage_good_windows_df)}, Retention Rate: {percentage_good_windows_df["Good Recording"].sum() / len(percentage_good_windows_df) * 100:.2f}%')
    plt.tight_layout(pad=0)
    plt.savefig(f'plots/signal quality/Percentage of Good Windows.png', dpi=dpi // 2)
    plt.close()

### Peak Power/SCI Over Time

In [14]:
if get_slope_pp_sci_plot:
    pp_channel_slopes = []
    sci_channel_slopes = []
    for pp_channel, sci_channel in zip(percentage_good_windows_peak_power_df.columns[4:], percentage_good_windows_sci_df.columns[4:]):
        pp_slope = []
        sci_slope = []
        for participant in peak_power_df['Participant'].unique():
            # get the data for the channel
            pp_array = peak_power_df[peak_power_df['Participant'] == participant][pp_channel]
            sci_array = sci_df[sci_df['Participant'] == participant][sci_channel]

            # get a line of best fit for the data
            x = np.arange(len(pp_array))
            pp_m, pp_b = np.polyfit(x, pp_array, 1)
            pp_slope.append(pp_m)

            sci_m, sci_b = np.polyfit(x, sci_array, 1)
            sci_slope.append(sci_m)
        pp_channel_slopes.append((pp_channel, np.mean(pp_slope)))
        sci_channel_slopes.append((sci_channel, np.mean(sci_slope)))

    # plot the slopes
    fig, ax = plt.subplots(figsize=(20, 6))
    # sort the channels by slope
    pp_channel_slopes.sort(key=lambda x: x[1])
    sci_channel_slopes.sort(key=lambda x: x[1])
    bar_width = 0.45
    x = np.arange(len(pp_channel_slopes))
    ax.bar(x, [slope for channel, slope in pp_channel_slopes], bar_width, label='Peak Power')
    ax.bar(x + bar_width, [slope for channel, slope in sci_channel_slopes], bar_width, label='SCI')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Slope')
    ax.set_title('Slope of Peak Power and SCI over Time')
    ax.set_xticks(x + bar_width / 2)
    ax.set_xticklabels([channel for channel, slope in pp_channel_slopes])
    ax.legend()
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'plots/signal quality/Slope of Peak Power and SCI over Time.png', dpi=dpi)
    plt.close()

### Head Size vs. SCI Windows

In [15]:
if get_head_size_vs_sci_plot:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    cap_size = 58
    percentage_good_windows_sci_df_with_head_size = percentage_good_windows_sci_df.copy()

    # add the head size to the percentage_good_windows_sci_df as the first column
    if 'Head Size (cm)' not in percentage_good_windows_sci_df_with_head_size.columns:
        percentage_good_windows_sci_df_with_head_size.insert(0, 'Head Size (cm)', cap_size)

    no_head_size = []

    for i, raw_haemo in enumerate(raw_haemos, 1):
        # get the head size
        head_size = get_info(raw_haemo)['remarks']
        if head_size:
            head_size = float(head_size) * 2.54
        else:
            # add the participant number to the no_head_size list
            no_head_size.append(i)
            continue
        
        # append the head_size to percentage_good_windows_sci_df
        percentage_good_windows_sci_df_with_head_size.loc[i, 'Head Size (cm)'] = head_size

    # remove the participants with no head size
    percentage_good_windows_sci_df_with_head_size = percentage_good_windows_sci_df_with_head_size.drop(no_head_size)

    # get the correlation between head size and the channels
    correlations = []
    for channel in percentage_good_windows_sci_df_with_head_size.columns[2:]:
        correlation = percentage_good_windows_sci_df_with_head_size['Head Size (cm)'].corr(percentage_good_windows_sci_df_with_head_size[channel])
        correlations.append((channel, correlation))

    # get the correlation between head size and the second column
    correlation = percentage_good_windows_sci_df_with_head_size['Head Size (cm)'].corr(percentage_good_windows_sci_df_with_head_size.iloc[:, 1])

    # plot the correlations
    fig, ax = plt.subplots(figsize=(20, 6))
    # sort the channels by correlation
    correlations.sort(key=lambda x: x[1])
    bar_width = 0.45
    x = np.arange(len(correlations))
    ax.bar(x, [correlation for channel, correlation in correlations], bar_width)
    ax.set_xlabel('Channel')
    ax.set_ylabel('Correlation')
    ax.set_title('Correlation between Head Size and SCI, Correlation with Good Recordings: ' + str(correlation) + ', N = ' + str(len(percentage_good_windows_sci_df_with_head_size)))
    ax.set_xticks(x)
    ax.set_xticklabels([channel for channel, correlation in correlations])
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'plots/signal quality/Correlation between Head Size and SCI.png', dpi=dpi / 3)
    plt.close()

C:\Users\super\AppData\Local\Temp\ipykernel_11268\2914967535.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '60.96' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  percentage_good_windows_sci_df_with_head_size.loc[i, 'Head Size (cm)'] = head_size


### Average SCI per Channel across Participants

In [16]:
if get_across_participant_sci_plots:
    # Load the percentage_good_windows_df
    percentage_good_windows_df = pd.read_csv('processed_data/windows/percentage_good_windows.csv')

    # get a list of participants in percentage_good_windows_sci_df where Good Recording is True
    good_participants = percentage_good_windows_df[percentage_good_windows_df['Good Recording'] == True].index

    # make a dataframe of the average sci for each channel
    avg_sci_df = sci_df.groupby('Participant').mean().drop(columns=['Window', 'Start_Time', 'End_Time'])

    # drop the participants that are not in good_participants
    avg_sci_df_good = avg_sci_df.loc[good_participants]

    # drop the participants that are in good_participants
    avg_sci_df_bad = avg_sci_df.drop(index=good_participants)

    # make a list of the dataframes
    avg_sci_dfs = [avg_sci_df, avg_sci_df_good, avg_sci_df_bad]
    df_names = ['All Participants', 'Good Participants', 'Bad Participants']
    color_list = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']

    for df, df_name in zip(avg_sci_dfs, df_names):

        # make a violin plot of the average sci for each channel
        fig, ax = plt.subplots(figsize=(35, 6))
        parts = ax.violinplot(df, showmeans=False, widths=1, showextrema=False)

        # match the violin plot colors to the columns in avg_sci_df to the channels in ch_mapping_names
        color_i = 0
        colors = []
        region_labels = []

        # for each region in ch_mapping_names, apply the color to the channels in that region
        for region, channels in mappings['ch_mapping_names'].items():
            for channel in channels:
                if channel in df.columns:
                    colors.append(color_list[color_i])
                    region_labels.append(region)
            color_i += 1

        # set the colors of the violins
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors[i])
            pc.set_edgecolor('black')
            pc.set_alpha(1)

        # create a legend
        handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in list(dict.fromkeys(colors))]
        ax.legend(handles, list(dict.fromkeys(region_labels)), loc='lower left')

        # add a white scatter plot of the mean sci for each channel
        ax.scatter(np.arange(1, len(df.columns) + 1), df.mean(), color='white', zorder=3)

        ax.set_xlabel('Channel')
        ax.set_ylabel('Average SCI')
        ax.set_ylim(0, 1)
        ax.axhline(good_threshold, color='green', linestyle='--')
        ax.set_title(f'Average SCI per Channel: ({df_name}), N = {len(df)}')
        ax.set_xticks(np.arange(1, len(df.columns) + 1))
        ax.set_xticklabels(df.columns)
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f'plots/signal quality/Average SCI (Windowed) per Channel/{df_name}.png', dpi=dpi / 3)
        plt.close()

# Get Epoch Data

In [ ]:
if get_epoch_data:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings

    modes = ['all', 'face_type', 'emotion', 'face_type_emotion']
    conditions_list = {
        'all': ['Base', 'Blck'],
        'face_type': ['Real', 'Virt'],
        'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
        'face_type_emotion': ['Real_Joy', 'Real_Fear', 'Real_Anger', 'Real_Disgust', 'Real_Sadness', 'Real_Neutral', 'Real_Surprise',
                              'Virt_Joy', 'Virt_Fear', 'Virt_Anger', 'Virt_Disgust', 'Virt_Sadness', 'Virt_Neutral', 'Virt_Surprise']
    }

    for mode in modes:
        epochs = get_epochs(raw_haemo_good_recordings, mode, max_times, 'epochs')
        for condition in conditions_list[mode]:
            # get the epochs for each condition
            epochs_condition = epochs[condition]

            # save the epochs to a file
            np.save(f'processed_data/epochs/{mode}_{condition}_epochs.npy', epochs_condition)

# We now have epochs[mode][condition].shape = (n_participants, n_epochs, n_channel_types, n_channels, n_times)
epochs = {
    mode: {
        condition: np.load(f'processed_data/epochs/{mode}_{condition}_epochs.npy')
        for condition in conditions_list[mode]
    }
    for mode in modes
}

# GLM analysis

### Get Design Matrix Plot

In [17]:
modes = ['face_type']
conditions_list = {
    'face_type': ['Real', 'Virt']
}
channel_types = ['hbo', 'hbr', 'hbt']

if get_glm_design_matrix_plot:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for mode in modes:
        raw_haemos_annotated = get_epochs(raw_haemo_good_recordings, mode, max_times, 'annotated')
        for i, raw_haemo in enumerate(raw_haemos_annotated, 1):
            if i == 36:
                break
            elif i < 35:
                continue
            raw_haemo_annots = pick_channels(raw_haemo, channel_types)

            # Create events for nilearn
            conditions = raw_haemo_annots.annotations.description
            onsets = raw_haemo_annots.annotations.onset - raw_haemo_annots.first_time
            duration = raw_haemo_annots.annotations.duration
            events = pd.DataFrame(
                {"trial_type": conditions, "onset": onsets, "duration": duration}
            )

            design_matrix = make_first_level_design_matrix(
                raw_haemo_annots.times,
                events,
                drift_model="cosine",
                hrf_model="spm",
                high_pass=0.03125,  # The cutoff period (1/high_pass) should be set as the longest period between two trials of the same condition multiplied by 2
            )

    # Create a mask for the design matrix
    design_matrix = design_matrix.loc[:, design_matrix.columns.str.contains("Real|Virt")]

    # only keep the first 250 seconds of the design matrix
    design_matrix = design_matrix.iloc[:int(250 * raw_haemo_annots.info['sfreq']), :]
        
    # plot the design matrix
    fig, ax = plt.subplots(figsize=(12, 6))
    design_matrix.plot(ax=ax, legend=True, linewidth=4, alpha=1)
    plt.xlabel("Time (s)", fontsize=18)
    plt.ylabel("Amplitude (AU)", fontsize=18)
    plt.title(f"Sample Design Matrix for Participant, First 250s", fontsize=24)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout(pad=0)
    plt.savefig(f"plots/figures/Sample Design Matrix.png", dpi=dpi // 2)
    plt.close()

C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nilearn\glm\first_level\experimental_paradigm.py:113: FutureWarning: The provided callable <function sum at 0x00000210015F54E0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  COLUMN_DEFINING_EVENT_IDENTITY, sort=False).agg(STRATEGY).reset_index()


### Run GLM Analysis

In [2]:
modes = ['all', 'face_type', 'emotion', 'face_type_emotion']
conditions_list = {
    'all': ['Blck', 'Base'],
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
    'face_type_emotion': ['Real_Joy', 'Real_Fear', 'Real_Anger', 'Real_Disgust', 'Real_Sadness', 'Real_Neutral', 'Real_Surprise',
                          'Virt_Joy', 'Virt_Fear', 'Virt_Anger', 'Virt_Disgust', 'Virt_Sadness', 'Virt_Neutral', 'Virt_Surprise']
}
channel_types = ['hbo', 'hbr', 'hbt']

if get_glm_analysis:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for mode in modes:
        cha_df = pd.DataFrame()
        roi_df = pd.DataFrame()
        con_df = pd.DataFrame()
        raw_haemos_annotated = get_epochs(raw_haemo_good_recordings, mode, max_times, 'annotated')
        for i, raw_haemo in enumerate(raw_haemos_annotated, 1):
            raw_haemo_annots = pick_channels(raw_haemo, channel_types)

            # Create events for nilearn
            conditions = raw_haemo_annots.annotations.description
            onsets = raw_haemo_annots.annotations.onset - raw_haemo_annots.first_time
            duration = raw_haemo_annots.annotations.duration
            events = pd.DataFrame(
                {"trial_type": conditions, "onset": onsets, "duration": duration}
            )

            design_matrix = make_first_level_design_matrix(
                raw_haemo_annots.times,
                events,
                drift_model="cosine",
                hrf_model="spm",
                high_pass=0.03125,  # The cutoff period (1/high_pass) should be set as the longest period between two trials of the same condition multiplied by 2
            )
            
            # Run GLM
            glm_est = run_glm(raw_haemo_annots, design_matrix, n_jobs=n_jobs)

            cha = glm_est.to_dataframe()

            # in ch_mapping_all, for each list of channels in the dict, each string is formatted as 'S{number}_D{number} {hbo/hbr}', extract the number from the string and replace the string with [number, number]
            groups = {region: [[int(re.findall(r'\d+', channel)[0]), int(re.findall(r'\d+', channel)[1])] for channel in mappings['ch_mapping_all'][region]] for region in mappings['ch_mapping_all']}
            # apply picks_pair_to_idx to each region in groups
            for region in groups:
                groups[region] = picks_pair_to_idx(raw_haemo_annots, groups[region], on_missing='ignore')

            # Compute region of interest results from channel data
            roi = glm_est.to_dataframe_region_of_interest(
                groups, design_matrix.columns, demographic_info=True
            )

            # Define contrasts
            contrast_matrix = np.eye(design_matrix.shape[1])
            basic_conts = dict(
                [(column, contrast_matrix[j]) for j, column in enumerate(design_matrix.columns)]
            )
            contrasts = []
            unique_annots = np.unique(raw_haemo_annots.annotations.description).tolist()
            pairs = list(itertools.combinations(unique_annots, 2))
            # include the opposite of each pair
            pairs = pairs + [(pair[1], pair[0]) for pair in pairs]

            # Compute defined contrast pairs
            for pair in pairs:
                con = glm_est.compute_contrast(basic_conts[pair[0]] - basic_conts[pair[1]]).to_dataframe()
                con["contrast_pair"] = f"{pair[0]} - {pair[1]}"
                contrasts.append(con)

            # Add the participant ID to the dataframes
            roi["Participant"] = cha["Participant"] = i
            for con in contrasts:
                con["Participant"] = i

            # Convert to uM for nicer plotting below.
            cha["theta"] = [t * 1.0e6 for t in cha["theta"]]
            roi["theta"] = [t * 1.0e6 for t in roi["theta"]]
            for con in contrasts:
                con["effect"] = [t * 1.0e6 for t in con["effect"]]

            # Append the dataframes to the main dataframes
            cha_df = pd.concat([cha_df, cha])
            roi_df = pd.concat([roi_df, roi])
            for con in contrasts:
                con_df = pd.concat([con_df, con])

        # Apply FDR correction to all dataframes
        for df in [cha_df, roi_df, con_df]:
            if 'p_value' in df.columns:
                _, df['p_value_fdr'] = fdrcorrection(df['p_value'])

        cha_df.to_csv('processed_data/glm/cha/cha_df_' + mode + '.csv', index=False)
        roi_df.to_csv('processed_data/glm/roi/roi_df_' + mode + '.csv', index=False)
        con_df.to_csv('processed_data/glm/cons/con_df_' + mode + '.csv', index=False)

# load the dataframes
glm = {
    mode: {
        'cha': pd.read_csv(f'processed_data/glm/cha/cha_df_{mode}.csv'),
        'roi': pd.read_csv(f'processed_data/glm/roi/roi_df_{mode}.csv'),
        'con': pd.read_csv(f'processed_data/glm/cons/con_df_{mode}.csv')
    }
    for mode in modes
}

### Individual GLM Results

In [4]:
if get_ind_glm_plots:
    channel_type = ['hbt']
    for mode in modes:
        grp_results = glm[mode]['roi'].query(f"Condition in {conditions_list[mode]}")
        grp_results = grp_results.query(f"Chroma in {channel_type}")

        theta_min = grp_results['theta'].min()
        theta_max = grp_results['theta'].max()

        # clear any files in the plots/glm/individual folder
        for f in os.listdir('plots/glm/individual_' + mode):
            os.remove(os.path.join('plots/glm/individual_' + mode, f))

        for i in grp_results['Participant'].unique():
            fig, ax = plt.subplots(figsize=(12, 6))
            fig.suptitle(f'GLM Results for Participant {i}')

            sns.swarmplot(data=grp_results.query(f"Participant == {i} and Chroma == '{channel_type[0]}'"),
                  x='Condition', y='theta', hue='ROI', ax=ax, dodge=False)
            ax.set_title(f'{channel_type[0]}')
            ax.set_ylabel('Theta (uM)')
            ax.set_ylim(theta_min, theta_max)
            ax.set_xlabel('Condition')
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig(f'plots/glm/individual_{mode}/Participant {i}.png', dpi=dpi / 4)
            plt.close()

### Group GLM Results

In [5]:
if get_group_glm_plots:
    channel_type = ['hbt']
    for mode in modes:
        grp_results = glm[mode]['roi'].query(f"Condition in {conditions_list[mode]}")
        grp_results = grp_results.query(f"Chroma in {channel_type}")

        # Run a GLM model
        roi_model = mixedlm("theta ~ -1 + ROI:Condition:Chroma", grp_results, groups=grp_results["Participant"]).fit(method="nm")

        # Get the results of the model and put it in a csv file
        roi_model_results = statsmodels_to_results(roi_model)

        # Apply FDR correction to the p-values in roi_model_results
        _, roi_model_results['P>|z|_fdr'] = fdrcorrection(roi_model_results['P>|z|'])

        # plot the results of the model for 'hbt' channel type
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle(f'GLM Results for Group - {mode}')

        sns.swarmplot(data=roi_model_results.query(f"Chroma == '{channel_type[0]}'"),
              x='Condition', y='Coef.', hue='ROI', ax=ax, dodge=False)
        ax.set_title(f'{channel_type[0]}')
        ax.set_ylabel('Theta (uM)')
        ax.set_xlabel('Condition')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f'plots/glm/group_results/results_{mode}.png', dpi=dpi / 4)
        plt.close()

### Group Contrasts

In [4]:
if get_group_contrast_plots:
    channel_type = 'hbt'
    for mode in modes:
        pairs = list(itertools.combinations(conditions_list[mode], 2))
        for pair in pairs:
            con_summary = glm[mode]['con'].query(f"contrast_pair == '{pair[0]} - {pair[1]}'")

            if len(con_summary) == 0:
                print(f"No data for contrast {pair[0]} - {pair[1]}")
                continue

            raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos
            raw_haemo = pick_channels(raw_haemos[0].copy(), 'hbo')
            raw_haemo_chs = pick_channels(raw_haemos[0].copy(), channel_type).ch_names

            con_summary_channel = con_summary.query(f"Chroma in {[channel_type]}")

            # Run group level model and convert to dataframe
            con_model = mixedlm("effect ~ -1 + ch_name:Chroma", con_summary_channel, groups=con_summary_channel["Participant"]).fit(method="nm")

            # Get the results of the model
            con_model_df = statsmodels_to_results(con_model, raw_haemo_chs)
            
            # Apply FDR correction to the group-level p-values from the model
            _, con_model_df['p_value_fdr'] = fdrcorrection(con_model_df['P>|z|'])

            # Plot the topographic map
            fig, axes = plt.subplots(1, 2, figsize=(10, 6))
            fig.suptitle(f'Contrast: {pair[0]} > {pair[1]}', fontsize=24, x=0.4)

            # set the "Coef." value to near 0 for channels that are not significant
            coef = con_model_df.copy()
            coef.loc[coef['p_value_fdr'] > 0.05, 'Coef.'] /= 15

            norm = mpl.colors.Normalize(vmin=-3, vmax=3)

            # plot the topomap
            mne.viz.plot_topomap(coef["Coef."].to_numpy(), raw_haemo.info, extrapolate='head', show=False, axes=axes[0], sensors='ok', cnorm=norm)

            # make a plot of a colorbar without using plt.colorbar
            cb1 = mpl.colorbar.ColorbarBase(axes[1], cmap=mpl.cm.RdBu_r, norm=norm, orientation='vertical')
            cb1.set_label('Coefficient', fontsize=16)
            cb1.ax.set_aspect(3)

            # make the colorbar numbers larger
            for label in cb1.ax.get_yticklabels():
                label.set_fontsize(16)

            plt.tight_layout(pad=0, rect=[0, 0, 1, 0.95])
            plt.subplots_adjust(wspace=-0.5)

            if "Neutral" in pair:
                plt.savefig(f'plots/glm/contrasts/differences_neutral/Contrast_{pair[0]}-{pair[1]}.png', dpi=dpi / 2)
            else:
                plt.savefig(f'plots/glm/contrasts/differences/Contrast_{pair[0]}-{pair[1]}.png', dpi=dpi / 2)
            plt.close()

### Significant Group Contrast Table

In [ ]:
if get_sig_group_contrast_table:
    all_significant_contrasts = pd.DataFrame()

    # pick channel type
    channel_type = 'hbt'

    for mode in modes:
        if mode == 'all':
            continue
        pairs = list(itertools.combinations(conditions_list[mode], 2))
        for pair in pairs:
            con_summary = glm[mode]['con'].query(f"contrast_pair == '{pair[0]} - {pair[1]}'")

            if len(con_summary) == 0:
                print(f"No data for contrast {pair[0]} - {pair[1]}")
                continue

            raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos
            raw_haemo = pick_channels(raw_haemos[0].copy(), 'hbo')
            raw_haemo_chs = pick_channels(raw_haemos[0].copy(), channel_type).ch_names

            con_summary_channel = con_summary.query(f"Chroma in {[channel_type]}")

            # Run group level model and convert to dataframe
            con_model = mixedlm("effect ~ -1 + ch_name:Chroma", con_summary_channel, groups=con_summary_channel["Participant"]).fit(method="nm")

            # Get the results of the model
            con_model_df = statsmodels_to_results(con_model, raw_haemo_chs)

            # Apply FDR correction to the group-level p-values from the model
            _, con_model_df['p_value_fdr'] = fdrcorrection(con_model_df['P>|z|'])

            # Add a column for the contrast pair
            con_model_df['contrast_pair'] = f"{pair[0]} > {pair[1]}"

            con_model_df['mode'] = mode

            # append the dataframe to the all_significant_contrasts dataframe
            filtered_con_model_df = con_model_df[con_model_df['p_value_fdr'] < 0.05]
            
            all_significant_contrasts = pd.concat([all_significant_contrasts, filtered_con_model_df])

            all_significant_contrasts.reset_index(drop=True, inplace=True)

            # Create a new column that matches ch_name to mappings['ch_mapping_hbt'] by finding the key in the dictionary
            all_significant_contrasts['Region'] = all_significant_contrasts['ch_name'].apply(lambda x: next((k for k, v in mappings['ch_mapping_hbt'].items() if x in v), None))

    all_significant_contrasts.to_csv('processed_data/glm/all_significant_contrasts.csv', index=False)

C:\Users\super\AppData\Local\Temp\ipykernel_31308\1331308252.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_significant_contrasts = pd.concat([all_significant_contrasts, filtered_con_model_df])
C:\Users\super\AppData\Local\Temp\ipykernel_31308\1331308252.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_significant_contrasts = pd.concat([all_significant_contrasts, filtered_con_model_df])
C:\Users\super\AppData\Local\Temp\ipykernel_31308\1331308252.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated.

# ROI Timeseries Activity

In [ ]:
if get_roi_timeseries_activity:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    epoch_dfs = get_epochs(raw_haemo_good_recordings, 'face_type_emotion', max_times, 'data')

    channel_types = ['hbo', 'hbr', 'hbt']

    roi_timeseries_activity = pd.DataFrame()

    onset_time = 4

    for i in range(len(epoch_dfs)):
        sex = epoch_dfs[i]['Sex'].unique()[0]
        # remove the sex column from the dataframe
        epoch_dfs[i] = epoch_dfs[i].drop(columns=['Sex'])

        # add a column for the participant number, make it the first column
        epoch_dfs[i].insert(0, 'Participant', i + 1)

        # for every "" in the description, replace it with the previous non "" value
        epoch_dfs[i]['description'] = epoch_dfs[i]['description'].replace('', np.nan).ffill()

        # Rename the time column to 'Time'
        epoch_dfs[i].rename(columns={'time': 'Time'}, inplace=True)

        # Split the description into two columns, one for the face type and one for the emotion
        epoch_dfs[i].insert(2, 'Face_type', epoch_dfs[i]['description'].str.split('_').str[0])
        epoch_dfs[i].insert(3, 'Emotion', epoch_dfs[i]['description'].str.split('_').str[1])

        # Remove the description column
        epoch_dfs[i] = epoch_dfs[i].drop(columns=['description'])

        # Initialize Epoch column in the 3rd position with NaN
        epoch_dfs[i].insert(2, 'Epoch', np.nan)

        # Assign epochs based on contiguous runs of (Face_type, Emotion) for non-Base/Task rows
        epoch_counter = 0
        is_nan = True

        for idx, row in epoch_dfs[i].iterrows():
            ft, emo = row['Face_type'], row['Emotion']
            if ft in ['Base', 'Task']:
                is_nan = True
                continue
            if pd.isna(emo):
                is_nan = True
                continue
            if emo != np.nan and is_nan:
                epoch_counter += 1
                is_nan = False
            # Assign epoch number to the current row
            epoch_dfs[i].at[idx, 'Epoch'] = epoch_counter

        # Remove the Emotion rows that are nan
        epoch_dfs[i] = epoch_dfs[i][epoch_dfs[i]['Emotion'].notna()]

        # get the min and max Time for each epoch and add the difference to a Duration column
        epoch_dfs[i].insert(5, 'Duration', np.nan)
        for epoch in epoch_dfs[i]['Epoch'].unique():
            epoch_df = epoch_dfs[i][epoch_dfs[i]['Epoch'] == epoch]
            min_time = epoch_df['Time'].min()
            max_time = epoch_df['Time'].max()
            duration = max_time - min_time
            epoch_dfs[i].loc[epoch_dfs[i]['Epoch'] == epoch, 'Duration'] = duration

        # remove the first onset_time seconds of each epoch by getting the time of the first index of each epoch and removing rows where that time + onset_time seconds is less than the Time column
        for epoch in epoch_dfs[i]['Epoch'].unique():
            epoch_df = epoch_dfs[i][epoch_dfs[i]['Epoch'] == epoch]
            min_time = epoch_df['Time'].min()
            epoch_dfs[i] = epoch_dfs[i][~((epoch_dfs[i]['Epoch'] == epoch) & (epoch_dfs[i]['Time'] < min_time + onset_time))]

        # Remove the Time column
        epoch_dfs[i] = epoch_dfs[i].drop(columns=['Time'])

        # Average the data for each epoch
        epoch_dfs[i] = epoch_dfs[i].groupby(['Participant', 'Epoch', 'Face_type', 'Emotion']).mean().reset_index()

        if 'hbo' in channel_types:
            for region, channels in mappings['ch_mapping_hbo'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in epoch_dfs[i].columns]
                if valid_channels:
                    # Create a new column for the region's average
                    epoch_dfs[i][region + ' Average Hbo'] = epoch_dfs[i][valid_channels].mean(axis=1)

        if 'hbr' in channel_types:
            for region, channels in mappings['ch_mapping_hbr'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in epoch_dfs[i].columns]
                if valid_channels:
                    # Create a new column for the region's average
                    epoch_dfs[i][region + ' Average Hbr'] = epoch_dfs[i][valid_channels].mean(axis=1)

        if 'hbt' in channel_types:
            for region, channels in mappings['ch_mapping_hbt'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in epoch_dfs[i].columns]
                if valid_channels:
                    # Create a new column for the region's average
                    epoch_dfs[i][region + ' Average Hbt'] = epoch_dfs[i][valid_channels].mean(axis=1)

        # drop all the channel columns
        epoch_dfs[i].drop(columns=mappings['all_channels'], inplace=True)

        # Add an empty column for repetition and put it after the Emotion column
        epoch_dfs[i].insert(4, 'Repetition', np.nan)

        conditions = defaultdict(int)
        for index, row in epoch_dfs[i].iterrows():
            # Add the Face_type-Emotion pair to the conditions dictionary and increment the count
            conditions[f"{row['Face_type']}-{row['Emotion']}"] += 1

            # Add the count to the Repetition column
            epoch_dfs[i].at[index, 'Repetition'] = conditions[f"{row['Face_type']}-{row['Emotion']}"]

        epoch_dfs[i].insert(5, 'Sex', sex)

        # add the dataframe to the average_timeseries_activity datafram
        roi_timeseries_activity = pd.concat([roi_timeseries_activity, epoch_dfs[i]])

    # reset the index
    roi_timeseries_activity.reset_index(drop=True, inplace=True)

    # name the index column to 'observation'
    roi_timeseries_activity.index.name = 'Observation'

    # replace any spaces in the column names with underscores
    roi_timeseries_activity.columns = roi_timeseries_activity.columns.str.replace(' ', '_')

    # capitalize the column names
    roi_timeseries_activity.columns = roi_timeseries_activity.columns.str.capitalize()

    mappings_file = {}
    for col in roi_timeseries_activity.select_dtypes(include=['object']).columns:
        # Create a mapping dictionary for the column
        unique_values = roi_timeseries_activity[col].unique()
        col_mapping = {val: idx for idx, val in enumerate(unique_values)}
        mappings_file[col] = col_mapping

        # Replace the column values in the DataFrame with numeric values
        roi_timeseries_activity[col] = roi_timeseries_activity[col].map(col_mapping)

    # Save mappings to a JSON file
    with open('processed_data/roi_timeseries_activity/mappings.json', 'w') as json_file:
        json.dump(mappings_file, json_file, indent=4)

    # Get the unique number of participants
    num_participants = roi_timeseries_activity['Participant'].nunique()

    # save the dataframe to a csv file
    roi_timeseries_activity.to_csv(f'processed_data/roi_timeseries_activity/roi_timeseries_activity_sci{sci_threshold}_n{num_participants}.csv')

# ERP Plots

In [2]:
modes = ['face_type', 'emotion', 'face_type_emotion']
conditions_list = {
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
    'face_type_emotion': ['Real_Joy', 'Real_Fear', 'Real_Anger', 'Real_Disgust', 'Real_Sadness', 'Real_Neutral', 'Real_Surprise',
                          'Virt_Joy', 'Virt_Fear', 'Virt_Anger', 'Virt_Disgust', 'Virt_Sadness', 'Virt_Neutral', 'Virt_Surprise']
}
channel_types = ['hbo', 'hbr', 'hbt']

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

if get_erp_plots:
    # We now have epochs[mode][condition].shape = (n_participants, n_epochs, n_channel_types, n_channels, n_times)
    epochs = {
        mode: {
            condition: np.load(f'processed_data/epochs/{mode}_{condition}_epochs.npy')
            for condition in conditions_list[mode]
        }
        for mode in modes
    }

    # make a color list for the different channel types
    colors = {
        'hbo': 'red',
        'hbr': 'blue',
        'hbt': 'purple'
    }

    # pick a channel type to plot the difference between regions
    difference_channel_type = 'hbt'

    for mode in modes:
        for condition in conditions_list[mode]:

            fig, ax = plt.subplots(figsize=(12, 6))

            for channel_type in channel_types:
                # get epoch data, shape is now (n_participants, n_times)
                epoch_data = np.nanmean(epochs[mode][condition][:, :, dict_channel_types[channel_type]], axis=(1, 2))

                # get the mean of the data
                epoch_data_mean = np.nanmean(epoch_data, axis=0)
                
                # get the standard error of the mean
                epoch_data_sem = np.nanstd(epoch_data, axis=0) / np.sqrt(epoch_data.shape[0])

                # plot the mean
                ax.plot(epoch_data_mean, label=channel_type, color=colors[channel_type])

                # plot the standard error of the mean as a shaded area
                ax.fill_between(np.arange(epoch_data_mean.shape[0]), epoch_data_mean - epoch_data_sem, epoch_data_mean + epoch_data_sem, color=colors[channel_type], alpha=0.3)
            
            ax.set_xlabel('Time (s)', fontsize=20)
            ax.set_ylabel('uM', fontsize=20)
            ax.set_title(f'Averaged Epochs for {mode}: {condition}', fontsize=24)
            ax.legend(fontsize=20)
            ax.set_ylim(-3e-6, 3e-6) # set the ylims to -+3 * 10^-6
            
            # set the xticks to the time points, 0-18 seconds, instead of 103 time points
            ax.set_xticks(np.arange(0, 103, 6))
            ax.set_xticklabels(np.arange(0, 103, 6) // 6)  # Ensure the number of labels matches the number of ticks
            # set the fontsize of the xticks to 20
            ax.tick_params(axis='x', labelsize=20)
            # set the fontsize of the yticks to 20
            ax.tick_params(axis='y', labelsize=20)
            plt.tight_layout(pad=0)
            plt.savefig(f'plots/erp/erp_conditions/{condition}.png', dpi=dpi / 2)
            plt.close()

        for cond1, cond2 in itertools.combinations(conditions_list[mode], 2):
            # create a 8x1 subplot that shares the x-axis and i can iterate over
            fig, axes = plt.subplots(8, 1, figsize=(12, 24), sharex=True)        

            # in ch_mapping_all, for each list of channels in the dict, filter for the channels with difference_channel_type in the string
            groups = {region: [channel for channel in channels if difference_channel_type in channel] for region, channels in mappings['ch_mapping_all'].items()}
            group_i = 0
            for region, ch_name in groups.items():
                # Get the indices for the current region from group_boundaries
                keys_list = list(groups.keys())
                start_idx = mappings['group_boundaries'][keys_list.index(region)]
                end_idx = mappings['group_boundaries'][keys_list.index(region) + 1] if keys_list.index(region) + 1 < len(mappings['group_boundaries']) else None

                # get the data for the current region
                epoch_data_region1 = np.nanmean(epochs[mode][cond1][:, :, dict_channel_types[difference_channel_type], start_idx:end_idx], axis=(1, 2))
                epoch_data_region2 = np.nanmean(epochs[mode][cond2][:, :, dict_channel_types[difference_channel_type], start_idx:end_idx], axis=(1, 2))

                # get the mean of the data
                epoch_data_region1_mean = np.nanmean(epoch_data_region1, axis=0)
                epoch_data_region2_mean = np.nanmean(epoch_data_region2, axis=0)

                # get the standard error of the mean
                epoch_data_region1_sem = np.nanstd(epoch_data_region1, axis=0) / np.sqrt(epoch_data_region1.shape[0])
                epoch_data_region2_sem = np.nanstd(epoch_data_region2, axis=0) / np.sqrt(epoch_data_region2.shape[0])

                # plot the mean
                axes[group_i].plot(epoch_data_region1_mean, label=cond1, color='red')
                axes[group_i].plot(epoch_data_region2_mean, label=cond2, color='blue')

                # plot the standard error of the mean as a shaded area
                axes[group_i].fill_between(np.arange(epoch_data_region1_mean.shape[0]), epoch_data_region1_mean - epoch_data_region1_sem, epoch_data_region1_mean + epoch_data_region1_sem, color='red', alpha=0.3)
                axes[group_i].fill_between(np.arange(epoch_data_region2_mean.shape[0]), epoch_data_region2_mean - epoch_data_region2_sem, epoch_data_region2_mean + epoch_data_region2_sem, color='blue', alpha=0.3)

                # set the title of the subplot to the region
                if group_i == 0:
                    axes[group_i].set_title(f'Averaged Epochs by ROI for {mode}: {cond1} - {cond2}, {difference_channel_type}\n{region}', fontsize=24)
                else:
                    axes[group_i].set_title(region, fontsize=24)
                
                # set the ylims to -+7 * 10^-6
                axes[group_i].set_ylim(-7e-6, 7e-6)

                # add a legend to the subplot
                axes[group_i].legend(fontsize=20)

                group_i += 1

            axes[-1].set_xlabel('Time (s)', fontsize=20)
            # set the xticks to the time points, 0-18 seconds, instead of 103 time points
            axes[-1].set_xticks(np.arange(0, 103, 6))
            axes[-1].set_xticklabels(np.arange(0, 103, 6) // 6)  # Ensure the number of labels matches the number of ticks
            # set the fontsize of the xticks to 20
            axes[-1].tick_params(axis='x', labelsize=20)
            # set the fontsize of the yticks to 20
            axes[-1].tick_params(axis='y', labelsize=20)
            axes[-1].set_ylabel('uM', fontsize=20)
            plt.tight_layout(pad=0)
            if "Neutral" == cond1 or "Neutral" == cond2:
                plt.savefig(f'plots/erp/erp_differences_neutral/{cond1}_{cond2}.png', dpi=dpi / 2)
            else:
                plt.savefig(f'plots/erp/erp_differences/{cond1}_{cond2}.png', dpi=dpi / 2)
            plt.close()

C:\Users\super\AppData\Local\Temp\ipykernel_28344\2364478111.py:41: RuntimeWarning: Mean of empty slice
  epoch_data = np.nanmean(epochs[mode][condition][:, :, dict_channel_types[channel_type]], axis=(1, 2))
C:\Users\super\AppData\Local\Temp\ipykernel_28344\2364478111.py:41: RuntimeWarning: Mean of empty slice
  epoch_data = np.nanmean(epochs[mode][condition][:, :, dict_channel_types[channel_type]], axis=(1, 2))
C:\Users\super\AppData\Local\Temp\ipykernel_28344\2364478111.py:44: RuntimeWarning: Mean of empty slice
  epoch_data_mean = np.nanmean(epoch_data, axis=0)
C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\super\AppData\Local\Temp\ipykernel_28344\2364478111.py:86: RuntimeWarning: Mean of empty slice
  epoch_data_region1 = np.nanmean(e

# Connectivity Analysis

In [2]:
modes = ['all', 'face_type', 'emotion', 'face_type_emotion']
conditions_list = {
    'all': ['Blck', 'Base'],
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
    'face_type_emotion': ['Real_Joy', 'Real_Fear', 'Real_Anger', 'Real_Disgust', 'Real_Sadness', 'Real_Neutral', 'Real_Surprise',
                          'Virt_Joy', 'Virt_Fear', 'Virt_Anger', 'Virt_Disgust', 'Virt_Sadness', 'Virt_Neutral', 'Virt_Surprise']
}

connectivity_method = [spectral_connectivity_time]

# pick the channels
channel_types = ['hbo', 'hbr', 'hbt']

# make a dictionary to store the channel types and their indices
dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

# main channel type
channel_type = 'hbt'

# sampling frequency
sfreq = 6.105006105006105

# pick the connectivity method
method = "coh"

# pick the mode
con_mode = "cwt_morlet"

# pick the frequency range
cwt_freqs = np.linspace(0.2, 0.5, 5)

# pick the number of cycles
cwt_n_cycles = 1

# average the connectivity matrices across frequencies
faverage = True

if run_ind_connectivity:
    # We now have epochs[mode][condition].shape = (n_participants, n_epochs, n_channel_types, n_channels, n_times)
    epochs = {
        mode: {
            condition: np.load(f'processed_data/epochs/{mode}_{condition}_epochs.npy')
            for condition in conditions_list[mode]
        }
        for mode in modes
    }

    # for each connectivity method, mode and condition, calculate the connectivity for each participant
    for func in connectivity_method:
        for mode in modes:
            for condition in conditions_list[mode]:
                # create an empty list to store the connectivity for each participant
                participant_cons = []

                for participant in range(epochs[mode][condition].shape[0]):

                    # create an empty list to store the connectivity for each channel type
                    channel_cons = []
                    
                    for channel_type in channel_types:
                        # get the data for the current participant and channel type
                        epoch_data = epochs[mode][condition][participant, :, dict_channel_types[channel_type], :]

                        # get the maximum number of time points across all epochs without NaN values
                        n_times_max = int(np.nanmax(np.sum(~np.isnan(epoch_data[:, 0, :]), axis=1)))

                        # reshape the data to (n_epochs, n_channels, n_times_max)
                        epoch_data = epoch_data[:, :, :n_times_max]

                        if func.__name__ == 'spectral_connectivity_time':
                            con = func(
                                epoch_data,
                                method=method,
                                mode=con_mode,
                                freqs=cwt_freqs,
                                n_cycles=cwt_n_cycles,
                                faverage=faverage,
                                sfreq=sfreq,
                                n_jobs=n_jobs,
                                verbose=False
                            )
                        else:
                            raise ValueError(f"Unknown connectivity method: {func.__name__}")

                        data = con.get_data()

                        channel_cons.append(data)

                    participant_cons.append(np.array(channel_cons))

                # save the connectivity to a file
                np.save(f'processed_data\\{func.__name__}\\individual_cons\\{mode}_{condition}_con.npy', np.array(participant_cons))

        # make a dictionary to store the connectivity parameters
        ind_connectivity_params = {
            "func": func.__name__,
            "channel_types": channel_types,
            "method": method,
            "con_mode": con_mode,
            "cwt_freqs": cwt_freqs.tolist(),
            "cwt_n_cycles": cwt_n_cycles,
            "faverage": faverage
        }

        # save the connectivity parameters to disk in preprocessed_data\connectivity
        with open(f"processed_data\\{func.__name__}\\ind_connectivity_params.json", "w") as f:
            json.dump(ind_connectivity_params, f)

# load the numpy files from disk so we have ind_con[func_name][mode][condition].shape = (n_participants, n_channel_types, n_epochs, n_channels, n_freqs)
ind_con = {
    func.__name__: {
        mode: {
            condition: np.load(f'processed_data\\{func.__name__}\\individual_cons\\{mode}_{condition}_con.npy')
            for condition in conditions_list[mode]
        }
        for mode in modes
    }
    for func in connectivity_method
}

ind_connectivity_params = {
    func.__name__: json.load(open(f"processed_data\\{func.__name__}\\ind_connectivity_params.json", "r"))
    for func in connectivity_method
}

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

participants = ind_con[connectivity_method[0].__name__][modes[0]][conditions_list[modes[0]][0]].shape[0]

### Condition Connectivity Heatmap/Chord Plots

In [3]:
if get_condition_con_plots:

    # make a color scheme
    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='hot',
        facecolor2='black',
        textcolor2='white',
        node_colors=['lightgrey'] * len(mappings['ch_names_original'])
    )

    # get the node angles
    node_angles = circular_layout(
        mappings['ch_names_original'], mappings['all_channels_names'], start_pos=90, group_boundaries=mappings['group_boundaries']
    )

    for func in connectivity_method:
        for mode in modes:
            for condition in conditions_list[mode]:
                averaged_data = np.nanmean(ind_con[func.__name__][mode][condition][:, dict_channel_types[channel_type]], axis=(0, 1, 3))
                
                # Get the grid size
                grid_size = int(np.sqrt(averaged_data.size))

                # Reshape the data into a 2D grid
                heatmap_data = averaged_data.reshape((grid_size, grid_size))

                # Make the matrix symmetric
                symmetric_data = heatmap_data + heatmap_data.T

                # Set the diagonal to the highest value
                np.fill_diagonal(symmetric_data, np.max(symmetric_data))

                # Plot the heatmap
                fig, ax = plt.subplots(figsize=(25, 25))
                im = ax.imshow(symmetric_data, cmap='viridis')
                ax.set_title(f'Connectivity Heatmap for {func.__name__}, Mode: {mode}, Condition: {condition}, Channel Type: {channel_type}')
                ax.set_xlabel('Channel')
                ax.set_ylabel('Channel')
                ax.set_xticks(np.arange(grid_size))
                ax.set_yticks(np.arange(grid_size))
                ax.set_xticklabels(mappings['all_channels_names'])
                ax.set_yticklabels(mappings['all_channels_names'])
                plt.setp(ax.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

                cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.set_label('Connectivity Value')
                plt.savefig(f'plots/{func.__name__}/heatmaps/conditions/{mode}_{condition}_con.png', dpi=dpi / 4)
                plt.close()

                # find the min of the averaged_data, when dropping all the 0s
                min_val = np.min(averaged_data[np.nonzero(averaged_data)])

                # find the max of the averaged_data, when dropping all the 0s
                max_val = np.max(averaged_data[np.nonzero(averaged_data)])

                # Plot the connectivity circle
                plot_connectivity_circle(
                    heatmap_data,
                    node_names=mappings['ch_names_original'],
                    node_angles=node_angles,
                    n_lines=10000,
                    colorbar_size=1,
                    fontsize_colorbar=16,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=colorscheme['colormap'],
                    node_colors=colorscheme['node_colors'],
                    padding=3,
                    vmin=min_val,
                    vmax=max_val,
                    colorbar=True,
                    show=False
                )
                plt.title(f'Connectivity: {condition}, {channel_type}', fontsize=24)
                plt.savefig(f'plots/{func.__name__}/chord_plots/conditions/{mode}_{condition}_con.png', dpi=dpi / 4)
                plt.close()

### Condition Connectivity Variance Across Participants

In [4]:
if get_variance_con_plots:
    for mode in modes:
        vmax = None
        for condition in conditions_list[mode]:
            # the shape of averaged_data is (n_participants, n_channels)
            averaged_data = np.nanmean(ind_con['spectral_connectivity_time'][mode][condition][:, dict_channel_types[channel_type], :, :, 0], axis=1)

            # reshape the data to (n_participants, n_channels, n_channels)
            averaged_data = averaged_data.reshape(averaged_data.shape[0], int(np.sqrt(averaged_data.shape[1])), int(np.sqrt(averaged_data.shape[1])))

            # make the matrix symmetric for the (sqrt(n_channels), sqrt(n_channels)) part
            for matrix in averaged_data:
                matrix += matrix.T
                np.fill_diagonal(matrix, np.nanmax(matrix))

            # create a matrix of the variance of each channel pair over all participants
            variance_matrix = np.nanvar(averaged_data, axis=0)

            # Plot the heatmap
            fig, ax = plt.subplots(figsize=(10, 10))
            im = ax.imshow(variance_matrix, cmap='viridis')
            ax.set_title(f'Variance Heatmap for Mode: {mode}, Condition: {condition}, Channel Type: {channel_type}')
            ax.set_xlabel('Channel')
            ax.set_ylabel('Channel')
            
            cbar = plt.colorbar(im, ax=ax)
            cbar.set_label('Variance Value')
            if vmax is None:
                vmax = variance_matrix.max()
            
            im.set_clim(0, vmax)
            plt.tight_layout()

            plt.savefig(f'plots/spectral_connectivity_time/heatmaps/variance/{mode}_{condition}_variance.png', dpi=dpi / 4)
            plt.close()

### Group Level Connectivity t-tests

In [3]:
if run_group_level_t_tests:
    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                # Get the data for the two conditions
                data1 = np.nanmean(ind_con[func.__name__][mode][condition1][:, dict_channel_types[channel_type]], axis=(1, 3))
                data2 = np.nanmean(ind_con[func.__name__][mode][condition2][:, dict_channel_types[channel_type]], axis=(1, 3))

                # reshape the data to (num_participants, int(np.sqrt(num_channel_connections)), int(np.sqrt(num_channel_connections)))
                data1 = data1.reshape(data1.shape[0], int(np.sqrt(data1.shape[1])), int(np.sqrt(data1.shape[1])))
                data2 = data2.reshape(data2.shape[0], int(np.sqrt(data2.shape[1])), int(np.sqrt(data2.shape[1])))

                # ensure the data shapes are the same
                assert data1.shape == data2.shape

                # ensure the data is not empty or contains NaNs
                assert not np.isnan(data1).any()
                assert not np.isnan(data2).any()

                # apply fisher z transform to the data
                data1 = np.arctanh(data1)
                data2 = np.arctanh(data2)

                # Initialize matrices to store t and p-values
                t_vals = np.zeros(data1.shape[1:])
                p_values = np.zeros(data1.shape[1:])

                # Iterate over each pair of channels
                for i in range(data1.shape[1]):
                    for j in range(i): # because the matrix is lower triangular
                        # Run the t-test
                        t_val, p_val = stats.ttest_rel(data1[:, i, j], data2[:, i, j])

                        # Store the results
                        t_vals[i, j] = t_val
                        p_values[i, j] = p_val

                # Flatten the lower triangular valid p-values (non-nan) for FDR correction
                valid_idx = ~np.isnan(p_values)
                p_values_flat = p_values[valid_idx]

                # Correct p-values for multiple comparisons
                _, p_values_flat_fdr = fdrcorrection(p_values_flat)

                # Reshape the corrected p-values back to matrix form
                p_values_fdr = p_values_flat_fdr.reshape(p_values.shape)

                # Save the p-values to disk
                np.save(f'processed_data\\{func.__name__}\\paired_t_tests_p_vals\\{mode}_{condition1}_{condition2}_p_values.npy', p_values_fdr)

                # Save the t-values to disk
                np.save(f'processed_data\\{func.__name__}\\paired_t_tests_t_vals\\{mode}_{condition1}_{condition2}_t_values.npy', t_vals)

# Load the t-values and p-values from disk and combine them into one dictionary
group_level_t_tests = {
    func.__name__: {
        mode: {
            f"{condition1}_{condition2}": {
                "p_vals": np.load(f'processed_data\\{func.__name__}\\paired_t_tests_p_vals\\{mode}_{condition1}_{condition2}_p_values.npy'),
                "t_vals": np.load(f'processed_data\\{func.__name__}\\paired_t_tests_t_vals\\{mode}_{condition1}_{condition2}_t_values.npy')
            }
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2)
        }
        for mode in modes
    }
    for func in connectivity_method
}

### Group Level Connectivity t-test Chord Plots


In [6]:
if get_group_level_t_tests_chord_plots:

    # make a color scheme
    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='berlin',
        facecolor2='black',
        textcolor2='white',
        node_colors=['lightgrey'] * len(mappings['ch_names_original'])
    )

    # get the node angles
    node_angles = circular_layout(
        mappings['ch_names_original'], mappings['all_channels_names'], start_pos=90, group_boundaries=mappings['group_boundaries']
    )
    
    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                t_values = group_level_t_tests[func.__name__][mode][f"{condition1}_{condition2}"]['t_vals']

                p_values = group_level_t_tests[func.__name__][mode][f"{condition1}_{condition2}"]['p_vals']

                # Set the t-values to 0 where the p-values are not significant
                t_values[p_values > 0.05] = 0

                # Set the diagonal to 0
                np.fill_diagonal(t_values, 0)

                plot_connectivity_circle(
                    t_values,
                    node_names=mappings['ch_names_original'],
                    node_angles=node_angles,
                    n_lines=10000,
                    colorbar_size=1,
                    fontsize_colorbar=16,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=colorscheme['colormap'],
                    node_colors=colorscheme['node_colors'],
                    padding=3,
                    vmin=-4,
                    vmax=4,
                    colorbar=True,
                    show=False
                )
                plt.title(f'Significantly different Channels: {condition1} > {condition2}, {channel_type}', fontsize=20)
                plt.savefig(f'plots/{func.__name__}/chord_plots/group_level_t_tests/{mode}_{condition1}_{condition2}.png', dpi=dpi / 4)
                plt.close()

### Group Level Connectivity t-test ROI Chord Plots
Remember to edit the plot_connectivity_circle function to include the diagonals: \mne\viz\circle.py

In [ ]:
if get_group_level_t_tests_roi_chord_plots:
    # make an empty dataframe to store the matrix data
    roi_p_vals = pd.DataFrame()
    roi_t_vals = pd.DataFrame()

    # Get the names of the regions
    regions = list(mappings['ch_mapping_names'].keys())

    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='bwr',
        facecolor2='black',
        textcolor2='white',
    )

    # get the node angles
    node_angles = circular_layout(
        regions, regions, start_pos=90
    )

    # First, flatten the dictionary into a list of tuples (channel_name, region)
    flat_list = [(channel, region) for region, channels in mappings['ch_mapping_names'].items() for channel in channels]

    # Now, replace the channel name with its enumeration (starting from 0)
    reshaped_list = [(i, region) for i, (_, region) in enumerate(flat_list)]

    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                t_values_by_roi = {}
                p_values_by_roi = {}

                for i, row in enumerate(group_level_t_tests[func.__name__][mode][f'{condition1}_{condition2}']['t_vals']):
                    region_i = reshaped_list[i][1]
                    for j, element in enumerate(row):
                        region_j = reshaped_list[j][1]

                        # skip the zeros
                        if element == 0:
                            continue

                        # Store the significant t-values and p-values for each ROI pair
                        if f'{region_i}_{region_j}' not in t_values_by_roi:
                            t_values_by_roi[f'{region_i}_{region_j}'] = [element]
                            p_values_by_roi[f'{region_i}_{region_j}'] = [group_level_t_tests[func.__name__][mode][f'{condition1}_{condition2}']['p_vals'][i][j]]
                        else:
                            t_values_by_roi[f'{region_i}_{region_j}'].append(element)
                            p_values_by_roi[f'{region_i}_{region_j}'].append(group_level_t_tests[func.__name__][mode][f'{condition1}_{condition2}']['p_vals'][i][j])

                for roi_pair, p_values in p_values_by_roi.items():
                    # set the t-values to 0 where the p-values are not significant
                    t_values_by_roi[roi_pair] = [t_value for t_value, p_value in zip(t_values_by_roi[roi_pair], p_values) if p_value < 0.05]

                    # Get the mean t-value for the ROI pair
                    t_values_by_roi[roi_pair] = np.mean(t_values_by_roi[roi_pair])

                    # filter out the p_values that are greater than 0.05
                    p_values_by_roi[roi_pair] = [p_value for p_value in p_values if p_value < 0.05]

                    # Get the count of p-values for each ROI pair
                    p_values_by_roi[roi_pair] = len(p_values_by_roi[roi_pair])

                # Create a matrix filled with NaNs
                t_matrix = np.full((len(regions), len(regions)), np.nan)

                # Create a matrix filled with NaNs for p-values
                p_matrix = np.full((len(regions), len(regions)), np.nan)

                # Fill the matrix using the dictionary values
                for key, value in t_values_by_roi.items():
                    r1, r2 = key.split('_')
                    i = regions.index(r1)
                    j = regions.index(r2)
                    # Fill the lower triangular part of the matrix
                    if i > j:
                        t_matrix[i, j] = value
                    else:
                        t_matrix[j, i] = value

                # Fill the p-values matrix using the dictionary values
                for key, value in p_values_by_roi.items():
                    r1, r2 = key.split('_')
                    i = regions.index(r1)
                    j = regions.index(r2)
                    # Fill the lower triangular part of the matrix
                    if i > j:
                        p_matrix[i, j] = value
                    else:
                        p_matrix[j, i] = value

                # Create dictionaries to hold the t-values and p-values data
                t_data_dict = {}
                p_data_dict = {}
                for i in range(len(regions)):
                    for j in range(len(regions)):
                        if not np.isnan(t_matrix[i, j]):
                            key = f"{regions[i]}_{regions[j]}"
                            t_data_dict[key] = t_matrix[i, j]
                        if not np.isnan(p_matrix[i, j]):
                            key = f"{regions[i]}_{regions[j]}"
                            p_data_dict[key] = p_matrix[i, j]

                # Create DataFrames from the dictionaries
                t_df = pd.DataFrame([t_data_dict])
                p_df = pd.DataFrame([p_data_dict])

                # Add the condition and mode as the first two columns in the DataFrames
                t_df.insert(0, 'Mode', mode)
                t_df.insert(1, 'Condition', f"{condition1}_{condition2}")
                p_df.insert(0, 'Mode', mode)
                p_df.insert(1, 'Condition', f"{condition1}_{condition2}")

                # Append the DataFrames to the matrix_data DataFrame
                roi_t_vals = pd.concat([roi_t_vals, t_df], ignore_index=True)
                roi_p_vals = pd.concat([roi_p_vals, p_df], ignore_index=True)

                # Plot the connectivity circle
                plot_connectivity_circle(
                    t_matrix,
                    node_names=regions,
                    node_angles=node_angles,
                    n_lines=10000,
                    colorbar_size=1,
                    fontsize_colorbar=16,
                    node_width=40,
                    linewidth=4.5,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=colorscheme['colormap'],
                    padding=3,
                    vmin=-4,
                    vmax=4,
                    fontsize_names=12,
                    colorbar=True,
                    show=False
                )
                plt.suptitle(f'Mean t-value by ROI: {condition1} > {condition2}\nMean t-value across ROI\'s: {np.round(np.nanmean(t_matrix), 2)}', fontsize=22)
                plt.savefig(f'plots/spectral_connectivity_time/chord_plots/group_level_t_tests_roi/{mode}_{condition1}_{condition2}.png', dpi=dpi / 2)
                plt.close()

    # Save the t-values and p-values DataFrames to CSV files
    roi_t_vals.to_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_t_vals.csv', index=False)
    roi_p_vals.to_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_p_vals.csv', index=False)

C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\super\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in sca

### Group Level Connectivity t-test ROI Emotion Analysis

In [19]:
if get_emotion_analysis_plots:
    # Read the CSV file into a DataFrame
    roi_t_vals = pd.read_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_t_vals.csv')
    roi_t_vals = roi_t_vals[roi_t_vals['Mode'] == 'emotion']

    # Split Condition into two columns and reorder them
    roi_t_vals = roi_t_vals.assign(
        Condition1=roi_t_vals['Condition'].str.split('_').str[0],
        Condition2=roi_t_vals['Condition'].str.split('_').str[1]
    ).drop(columns=['Condition'])

    # Reorder columns to place Condition1 and Condition2 after Mode
    cols = ['Mode', 'Condition1', 'Condition2'] + [col for col in roi_t_vals.columns if col not in ['Mode', 'Condition1', 'Condition2']]
    roi_t_vals = roi_t_vals[cols]

    # Get the unique conditions from Condition1 and Condition2
    unique_conditions = pd.Series(roi_t_vals['Condition1'].tolist() + roi_t_vals['Condition2'].tolist()).unique()

    # Prepare a matrix for the heatmap: rows and columns are unique_conditions, values are mean across all ROIs for each pair
    heatmap_matrix = np.full((len(unique_conditions), len(unique_conditions)), np.nan)
    for i, cond1 in enumerate(unique_conditions):
        for j, cond2 in enumerate(unique_conditions):
            if i >= j:  # Only fill lower triangle (including diagonal)
                mask = (
                    ((roi_t_vals['Condition1'] == cond1) & (roi_t_vals['Condition2'] == cond2)) |
                    ((roi_t_vals['Condition1'] == cond2) & (roi_t_vals['Condition2'] == cond1))
                )
                if mask.any():
                    # Drop Mode, Condition1, Condition2 columns, then mean across all ROI columns for this pair
                    vals = roi_t_vals.loc[mask].drop(columns=['Mode', 'Condition1', 'Condition2']).mean(axis=1)
                    heatmap_matrix[i, j] = vals.mean()

    # Set the diagonal to 0
    np.fill_diagonal(heatmap_matrix, 0)

    # Plot the heatmap
    plt.figure(figsize=(14, 12))
    im = plt.imshow(heatmap_matrix, cmap='bwr', aspect='auto', vmin=-3, vmax=3)
    cbar = plt.colorbar(im, label='Mean t-value')
    cbar.ax.tick_params(labelsize=20)  # Increase colorbar tick size
    cbar.set_label('Mean t-value', fontsize=24)  # Increase colorbar label size
    plt.xticks(ticks=np.arange(len(unique_conditions)), labels=unique_conditions, rotation=45, fontsize=20)
    plt.yticks(ticks=np.arange(len(unique_conditions)), labels=unique_conditions, fontsize=20)
    plt.xlabel('Emotion 1', fontsize=24)
    plt.ylabel('Emotion 2', fontsize=24)
    plt.title('Mean t-value by Emotion Pair across ROI\'s\nContrast: Emotion 1 > Emotion 2', fontsize=28)
    plt.tight_layout(pad=0)
    plt.savefig(f'plots/spectral_connectivity_time/emotion_analysis/Mean_t-value_by_Emotion_Pair.png', dpi=dpi / 2)
    plt.close()

    # Read the CSV file into a DataFrame
    roi_p_vals = pd.read_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_p_vals.csv')
    roi_p_vals = roi_p_vals[roi_p_vals['Mode'] == 'emotion']

    # Split Condition into two columns and reorder them
    roi_p_vals = roi_p_vals.assign(
        Condition1=roi_p_vals['Condition'].str.split('_').str[0],
        Condition2=roi_p_vals['Condition'].str.split('_').str[1]
    ).drop(columns=['Condition'])

    # Reorder columns to place Condition1 and Condition2 after Mode
    cols = ['Mode', 'Condition1', 'Condition2'] + [col for col in roi_p_vals.columns if col not in ['Mode', 'Condition1', 'Condition2']]
    roi_p_vals = roi_p_vals[cols]

    # Get the unique conditions from Condition1 and Condition2
    unique_conditions = pd.Series(roi_p_vals['Condition1'].tolist() + roi_p_vals['Condition2'].tolist()).unique()

    all_conditions = pd.DataFrame()
    for condition in unique_conditions:
        # Filter the DataFrame for the current condition in both Condition1 and Condition2
        condition_data = roi_p_vals[(roi_p_vals['Condition1'] == condition) | (roi_p_vals['Condition2'] == condition)]

        # drop the Mode column
        condition_data = condition_data.drop(columns=['Mode'])

        # Create a new column 'Condition' based on the current condition
        condition_data.insert(0, 'Condition', condition)

        # drop the Condition1 and Condition2 columns
        condition_data = condition_data.drop(columns=['Condition1', 'Condition2'])

        # average the data for the current condition
        condition_data = condition_data.groupby('Condition').mean().reset_index()

        # Concatenate the filtered DataFrame to all_conditions
        all_conditions = pd.concat([all_conditions, condition_data], ignore_index=True)

    # compute the mean across all columns except 'Condition'
    all_conditions['RegionSum'] = all_conditions.drop(columns='Condition').sum(axis=1)

    # keep only Condition + the new mean column
    all_conditions = all_conditions[['Condition', 'RegionSum']]

    # sort the dataframe by RegionMean
    all_conditions = all_conditions.sort_values(by='RegionSum', ascending=True)

    # reset the index
    all_conditions = all_conditions.reset_index(drop=True)

    # produce a bar plot of the mean values for each condition
    plt.figure(figsize=(16, 12))
    plt.barh(all_conditions['Condition'], all_conditions['RegionSum'], color=plt.cm.Wistia(np.linspace(0, 1, len(all_conditions))))
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.xlabel('Count of significantly different channel pairs', fontsize=24)
    plt.ylabel('Emotion', fontsize=20)
    plt.title('Count of significantly different channel pairs summed across all Regions,\nby Emotion', fontsize=24)
    plt.tight_layout()
    plt.savefig(f'plots/spectral_connectivity_time/emotion_analysis/Count_of_Significant_Channel_Pairs_by_Emotion.png', dpi=dpi / 2)
    plt.close()

    # Read the CSV file into a DataFrame
    roi_p_vals = pd.read_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_p_vals.csv')
    roi_p_vals = roi_p_vals[roi_p_vals['Mode'] == 'emotion']

    labels = roi_p_vals['Condition']

    # drop the Mode and Condition columns and sum the rest of the columns
    roi_p_vals_sum = roi_p_vals.drop(columns=['Mode', 'Condition']).sum(axis=0)

    # create a new dataframe with the regions as the index and the number of significant channels as the values
    roi_p_vals_df = pd.DataFrame(roi_p_vals_sum, columns=['Number of Significant Channels'])
    roi_p_vals_df.index.name = 'Region'
    roi_p_vals_df.reset_index(inplace=True)

    # split the Region column into two columns: Region 1 and Region 2
    roi_p_vals_df[['Region 1', 'Region 2']] = roi_p_vals_df['Region'].str.split('_', expand=True)
    roi_p_vals_df.drop(columns=['Region'], inplace=True)

    # create a matrix of the number of significant channels for each region pair
    matrix = roi_p_vals_df.pivot(index='Region 1', columns='Region 2', values='Number of Significant Channels')

    # fill the NaN values with the symmetric values
    matrix = matrix.fillna(matrix.T)

    # get rid of the upper triangle of the matrix by setting it to NaN
    for i in range(len(matrix.columns)):
        for j in range(i):
            matrix.iloc[j, i] = np.nan

    # plot the heatmap
    plt.figure(figsize=(14, 12))
    plt.imshow(matrix, cmap='Wistia', aspect='auto')
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=20)  # Increase colorbar tick size
    cbar.set_label('Count of channel pairs', fontsize=24)  # Increase colorbar label size
    plt.title('Count of significantly different channel pairs\n summed across all Emotions by ROI\'s', fontsize=28)
    plt.xticks(ticks=np.arange(len(matrix.columns)), labels=matrix.columns, rotation=45, fontsize=20)
    plt.yticks(ticks=np.arange(len(matrix.index)), labels=matrix.index, fontsize=20)
    plt.xlabel('ROI', fontsize=24)
    plt.ylabel('ROI', fontsize=24)

    # Find the locations of the smallest 3 values in the matrix
    flat_matrix = matrix.values.flatten()
    # Ignore NaNs for sorting
    valid_indices = np.where(~np.isnan(flat_matrix))[0]
    sorted_indices = valid_indices[np.argsort(flat_matrix[valid_indices])]
    least_3_indices = sorted_indices[:3]
    least_3_coords = np.unravel_index(least_3_indices, matrix.shape)

    # Add asterisks at the locations of the smallest 3 values
    for y, x in zip(*least_3_coords):
        plt.text(x, y, '*', color='black', fontsize=30, ha='center', va='center')

    plt.tight_layout(pad=0)
    plt.savefig(r'plots\spectral_connectivity_time\emotion_analysis\Count_of_Significant_Channel_Pairs_by_Region.png', dpi=dpi / 2)
    plt.close()

# Decoding Analysis

## Get Data

In [28]:
modes = ['face_type', 'emotion']

conditions_list = {
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

channel_types = ['hbo', 'hbr', 'hbt']

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

channel_type = 'hbt'

# We now have epochs[mode][condition].shape = (n_participants, n_epochs, n_channel_types, n_channels, n_times)
epochs = {
    mode: {
        condition: np.load(f'processed_data/epochs/{mode}_{condition}_epochs.npy')
        for condition in conditions_list[mode]
    }
    for mode in modes
}

# load the numpy files from disk so we have ind_con[mode][condition].shape = (n_participants, n_channel_types, n_epochs, n_channels)
ind_con = {
    mode: {
        condition: np.load(f'processed_data\\spectral_connectivity_time\\individual_cons\\{mode}_{condition}_con.npy')
        for condition in conditions_list[mode]
    }
    for mode in modes
}

## Across Participants

### Using Raw Data

#### Traditional Models

In [ ]:
if run_traditional_raw_across_decoding:
    models_to_run = [
        HistGradientBoostingClassifier(random_state=42)
    ]

    logo = LeaveOneGroupOut()

    for model in models_to_run:
        model_name = model.__class__.__name__
        all_mode_results = {}

        for mode in modes:
            # get the data for the current mode, shape = (n_participants, n_epochs, n_channels, n_times)
            X = np.concatenate([epochs[mode][condition][:, :, dict_channel_types[channel_type], :] for condition in conditions_list[mode]], axis=0)

            # get the labels for the current mode, shape = (n_participants * n_epochs,)
            y = np.concatenate([
                np.repeat(condition, epochs[mode][condition].shape[0] * epochs[mode][condition].shape[1])
                for condition in conditions_list[mode]
            ], axis=0)

            # get the group IDs for the current mode, shape = (n_participants * n_epochs,)
            groups = np.concatenate([
                np.repeat(participant, epochs[mode][condition].shape[1])
                for condition in conditions_list[mode]
                for participant in range(epochs[mode][condition].shape[0])
            ], axis=0)
            
            # flatten X to (n_samples, n_features)
            X = X.reshape(X.shape[0] * X.shape[1], X.shape[2] * X.shape[3])

            # --- parallel Leave-One-Group-Out evaluation ---
            pipe = make_pipeline(StandardScaler(), model)
            cv_results = cross_validate(
                estimator=pipe,
                X=X, y=y,
                groups=groups,
                cv=logo,
                scoring='accuracy',
                n_jobs=max(n_jobs // 3, 1),
                return_train_score=False, 
                verbose=1
            )

            # scale to percentages
            logo_scores = list(cv_results['test_score'] * 100)
            mean_score = float(np.mean(logo_scores))

            all_mode_results[mode] = {
                'logo_scores': logo_scores,
                'mean_score': mean_score
            }

        # write out
        out_path = f'processed_data/models/raw_across_scores/{model_name}_scores.json'
        with open(out_path, 'w') as f:
            json.dump(all_mode_results, f)

#### LSTM Model

In [ ]:
if run_dl_raw_across_decoding:
    # 1) Prepare cross‐validation splitter
    logo = LeaveOneGroupOut()
    results_by_mode = {}

    for mode in modes:
        # -------------------------
        # 2) Assemble data arrays
        # -------------------------
        # Stack all participants & conditions together:
        #   X_raw shape before concat: list of arrays, each (n_participants, n_epochs, n_channels, n_times)
        X_parts = [
            epochs[mode][cond][:, :, dict_channel_types[channel_type], :]
            for cond in conditions_list[mode]
        ]
        # X_parts: list of arrays, each (n_participants, n_epochs, n_channels, n_times)
        # Stack along participants and epochs
        # Reshape each to (n_participants * n_epochs, n_channels, n_times)
        X_parts_reshaped = [
            x.reshape(-1, x.shape[2], x.shape[3]) for x in X_parts
        ]
        X = np.concatenate(X_parts_reshaped, axis=0)  # shape = (n_samples, n_channels, n_times)

        # Build integer‐coded label vector y
        y_parts = []
        groups_parts = []
        for cond_idx, cond in enumerate(conditions_list[mode]):
            data = epochs[mode][cond][:, :, dict_channel_types[channel_type], :]
            n_participants, n_epochs, n_channels, n_times = data.shape

            # labels: one integer per epoch of each participant
            y_parts.append(
                np.full(n_participants * n_epochs, cond_idx, dtype=int)
            )
            # groups: participant ID repeated for each epoch
            groups_parts.append(
                np.repeat(np.arange(n_participants), n_epochs)
            )

        y = np.concatenate(y_parts, axis=0)             # shape = (n_samples,)
        groups = np.concatenate(groups_parts, axis=0)   # shape = (n_samples,)

        # One‐hot encode labels
        num_classes = len(conditions_list[mode])
        y_onehot = np.eye(num_classes, dtype=float)[y]  # shape = (n_samples, num_classes)

        # -------------------------
        # 3) Cross‐validate
        # -------------------------
        fold_accuracies = []
        for train_idx, test_idx in logo.split(X, y, groups):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y_onehot[train_idx], y_onehot[test_idx]

            # Build the model
            model = tf.keras.Sequential([
                tf.keras.layers.LSTM(32,
                                     input_shape=(X_train.shape[1], X_train.shape[2]),
                                     return_sequences=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(num_classes, activation='softmax')
            ])
            model.compile(optimizer='adam',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

            # Early stopping
            early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True,
                verbose=0
            )

            # Train
            model.fit(
                X_train, y_train,
                epochs=30,
                batch_size=32,
                validation_split=0.2,
                callbacks=[early_stop],
                verbose=1
            )

            # Evaluate
            _, acc = model.evaluate(X_test, y_test, verbose=0)
            fold_accuracies.append(acc * 100)

        # Store results
        results_by_mode[mode] = {
            'fold_accuracies': [float(a) for a in fold_accuracies],
            'mean_accuracy': float(np.mean(fold_accuracies)),
            'std_accuracy': float(np.std(fold_accuracies))
        }

    # -------------------------
    # 4) Save results to JSON
    # -------------------------
    output_path = (
        'processed_data/models/raw_across_scores/'
        f'{model.__class__.__name__}_scores.json'
    )
    # Ensure all values are native Python types
    with open(output_path, 'w') as out_file:
        json.dump(results_by_mode, out_file, indent=2)

ValueError: Found input variables with inconsistent numbers of samples: [104, 2912, 2912]

### Using Connectivity Data

#### Traditional Models

In [ ]:
if run_traditional_con_across_decoding:
    models_to_run = [
        HistGradientBoostingClassifier(random_state=42)
    ]

    logo = LeaveOneGroupOut()

    for model in models_to_run:
        model_name = model.__class__.__name__
        model_scores = {}

        for mode in modes:
            # --- build X, y, groups as before ---
            X_list, y_list, groups_list = [], [], []
            for i in range(ind_con[modes[0]][conditions_list[modes[0]][0]].shape[0]):
                parts_X, parts_y = [], []
                for cond in conditions_list[mode]:
                    x = ind_con[mode][cond][i, dict_channel_types[channel_type], :, :, 0]
                    parts_X.append(x)
                    parts_y.append(np.array([cond] * x.shape[0]))
                X_part = np.vstack(parts_X)
                y_part = np.hstack(parts_y)
                label_map = {lbl: idx for idx, lbl in enumerate(np.unique(y_part))}
                y_part = np.array([label_map[lbl] for lbl in y_part])
                X_list.append(X_part)
                y_list.append(y_part)
                groups_list.append(np.full(len(y_part), i))

            X = np.concatenate(X_list, axis=0)
            y = np.concatenate(y_list, axis=0)
            groups = np.concatenate(groups_list, axis=0)

            # --- cross_validate in parallel ---
            pipe = make_pipeline(StandardScaler(), model)
            cv_results = cross_validate(
                estimator=pipe,
                X=X, y=y,
                cv=logo,
                groups=groups,
                scoring='accuracy',
                n_jobs=max(n_jobs // 3, 1),
                return_train_score=False, 
                verbose=1
            )

            # convert to percentages
            test_scores = cv_results['test_score'] * 100
            model_scores[mode] = {
                'logo_scores': test_scores.tolist(),
                'mean_score': float(np.mean(test_scores))
            }

        # save out
        out_fname = f'processed_data/models/con_across_scores/{model_name}_scores.json'
        with open(out_fname, 'w') as f:
            json.dump(model_scores, f)

## Decoding Analysis Plots

### Accuracy Tables

In [2]:
score_types = ['raw_across_scores', 'con_across_scores']

if get_decoding_table_scores_plots:
    for score_type in score_types:
        # load data from 'processed_data/models' folder
        model_scores = {}
        # for any csv file in the folder
        for file in os.listdir(f'processed_data/models/{score_type}'):
            # if the file is a json file
            if file.endswith('.json'):
                # open the file
                with open(f'processed_data/models/{score_type}/{file}', 'r') as f:
                    # load the data from the file
                    model_scores[file.split('_')[0]] = json.load(f)

        # split score_type by '_'
        score_type_split = score_type.split('_')
        if score_type_split[0] == 'con':
            score_type_split[0] = 'Connectivity'
        elif score_type_split[0] == 'raw':
            score_type_split[0] = 'Raw'

        score_type_split[1] = score_type_split[1].capitalize()

        # Create a DataFrame from the model scores
        model_scores_df = pd.DataFrame([
            {'Model': model_name, 'Mode': mode, 'Mean Score': score['mean_score']}
            for model_name, scores in model_scores.items()
            for mode, score in scores.items()
        ])

        # Pivot the DataFrame to make 'Model' the first column and each 'Mode' a separate column
        model_scores_df = model_scores_df.pivot(index='Model', columns='Mode', values='Mean Score').reset_index()

        # Fill NaN values with 0 or any other value if needed
        model_scores_df.fillna(np.nan, inplace=True)

        # Remove the Mode column name
        model_scores_df.columns.name = None

        # Reorder the columns to have 'Model', 'face_type', 'emotion'
        model_scores_df = model_scores_df[['Model', 'face_type', 'emotion']]

        # Round the scores to 2 decimal places
        model_scores_df = model_scores_df.round(2)

        # Create a matplotlib figure and add a table
        fig, ax = plt.subplots(figsize=(6, len(model_scores_df) / 4))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=model_scores_df.values, colLabels=model_scores_df.columns, loc='center')

        # Set the biggest number in each column to be bold
        for i, col in enumerate(model_scores_df.columns):
            if col == 'Model':
                continue
            max_val = model_scores_df[col].max()
            for j, val in enumerate(model_scores_df[col]):
                if val == max_val:
                    table[(j + 1, i)].set_text_props(weight='bold')

        # Add a title to the plot
        ax.set_title(f'{score_type_split[0]} {score_type_split[1]} Scores (Accuracy%)')

        # Save the figure as a PNG file, bbox_inches='tight'
        plt.savefig(f'plots/models/tables/{score_type}.png', dpi=dpi, bbox_inches='tight')
        plt.close()

### Participant/LOGO Scores Plot

In [3]:
if get_decoding_individual_scores_plots:
    for score_type in score_types:
        # load data from 'processed_data/models' folder
        model_scores = {}
        # for any csv file in the folder
        for file in os.listdir(f'processed_data/models/{score_type}'):
            # if the file is a json file
            if file.endswith('.json'):
                # open the file
                with open(f'processed_data/models/{score_type}/{file}', 'r') as f:
                    # load the data from the file
                    model_scores[file.split('_')[0]] = json.load(f)

        # split score_type by '_'
        score_type_split = score_type.split('_')
        if score_type_split[0] == 'con':
            score_type_split[0] = 'Connectivity'
        elif score_type_split[0] == 'raw':
            score_type_split[0] = 'Raw'

        score_type_split[1] = score_type_split[1].capitalize()

        models = list(model_scores.keys())
        modes = list(model_scores[list(model_scores.keys())[0]].keys())
        
        fig, ax = plt.subplots(figsize=(10, 6))
        colors = plt.cm.tab10.colors  # Use a colormap for distinct colors

        # Assign a consistent color to each model
        model_colors = {model: colors[idx % len(colors)] for idx, model in enumerate(models)}

        for mode_idx, mode in enumerate(modes):
            for model_idx, model in enumerate(models):
                scores = list(model_scores[model][mode].keys())[0]
                vp = plt.violinplot(
                    model_scores[model][mode][scores],
                    positions=[model_idx + mode_idx * (len(models) + 1)],
                    showmeans=True,
                    showextrema=True,
                    widths=0.5,
                    bw_method=0.5,
                )
                # Set the color for the violin parts
                for part in vp['bodies']:
                    part.set_facecolor(model_colors[model])
                    part.set_edgecolor('black')
                    part.set_alpha(0.7)
                vp['cmeans'].set_color('black')  # Median line color
                vp['cbars'].set_color('black')    # Whisker line color
                vp['cmaxes'].set_color('black')   # Max line color
                vp['cmins'].set_color('black')    # Min line color

        # Set x-ticks and labels
        x_ticks = [(len(models) + 1) * i + len(models) / 2 - 0.5 for i in range(len(modes))]
        ax.set_xticks(x_ticks)
        ax.set_xticklabels(modes)
        ax.set_xlabel('Modes')
        ax.set_ylabel('Scores')
        ax.set_title(f'{score_type_split[0]} {score_type_split[1]} Scores (Accuracy%)')
        ax.set_ylim(0, 100)
        ax.set_yticks(np.arange(0, 101, 10))
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        ax.legend(
            handles=[mpatches.Patch(color=model_colors[model], label=model) for model in models],
            title='Models',
            loc='upper right'
        )
        plt.tight_layout()
        plt.savefig(f'plots/models/violin_plots/{score_type}.png', dpi=dpi / 2)
        plt.close()